# Part-1 CODE-BLOCK
- Do not make any changes in the 'CODE-BLOCK'
- This notebook retrieves the required data from specified websites with email addresses from the home page and all the link pages of the site, if available. It is a slower execution.
- Execute all cells in sequence
- This notebook may take up to 10 hours to run, so if you are executing all cells from Kernel, ensure availability of internet for this duration.
- Test-Blocks are disabled but for safety do not try to execute any cell marked as Test-Block

In [1]:
# import necessary libraries
import time
import requests
import re
import numpy as np
import pandas as pd
from contextlib import suppress
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# Install webdriver manager to automatically detect driver location
service = ChromeService(executable_path=ChromeDriverManager().install())

# Set up ChromeDriver options
chrome_options = Options()
chrome_options.add_argument('--headless') # Run in headless mode to avoid opening a new browser window


In [2]:
def extract_hrefs(n=2, sleep = 15):
    """
    Extracts all hrefs from a paginated web url as a list 
    Args:
        n = number of paginated web urls from which hrefs are to be extracted, default = 5
        sleep = time in seconds to sleep before quitting the driver, default = 15 sec
        url = paginated url from which data is to be extracted
    Returns:
        (list) containining the hrefs found on each page 
    
    """
    
        # Initialize empty list to store results
    all_hrefs = []

    
    # Loop over n list pages
    for i in range(1, n+1):
        # Build URL for current list pages having links to company profiles
        # Don't uncomment following line of code
        url = f"https://wirmarket.wir.ch/de/members/list/?page={i}&resultAmount=100"
        

        # Launch ChromeDriver and load websites
        # In order to disable browser windows opening for websites amend the code of the following line:
        # driver = webdriver.Chrome(service=service, options = chrome_options)
        driver = webdriver.Chrome(service=service)

        # Don't uncomment the code on the following line, it is for testing
        #   driver.implicitly_wait(20) # seconds
        driver.get(url)

        # Wait for all anchor elements to be present
        # WebDriverWait(driver, 180).until(EC.visiblity_of_all_elements_located((By.CSS_SELECTOR, "a")))
        # Due to intermittent functionality of WebDriverWait, we introduce sleep() function to get all anchors
        # This value is presently set at 15 secs but you may change it depending upon internet speed
        time.sleep(sleep)
        # Extract hrefs from all anchor elements on current website
        hrefs = [a.get_attribute('href') for a in driver.find_elements(By.CSS_SELECTOR, "a")]

        # Add hrefs to the list of all hrefs
        all_hrefs.extend(hrefs) # extend flattens the list so that it is no longer a list of lists

        # Close the current browser window
        driver.quit()
    return all_hrefs


In [33]:
### Test Block ! DO NOT EXECUTE
# url = f"https://wirmarket.wir.ch/de/members/list/?page={i}&resultAmount=100"
# all_hrefs_list = extract_hrefs(n=2, sleep = 15, 
#                          url = url)
                          
# # Check the length of all_hrefs list
# # Each list link contains between 400-500 hrefs
# print(len(all_hrefs_list))
# # Check for duplicates
# print(len(set(all_hrefs_list)))

# # Test for function extract_hrefs passed

In [3]:
def unique_urls(urls_list):
    """
    Removes repeated entries of urls in a list which might occur due to use of regex or multiple occurences on a page
    Args:
        urls_list: list containing urls which might be repetitive entries
    Returns: list containing unique urls
    """
    # convert to set
    urls_set = set(urls_list)
    # convert back to list
    unique_urls_list = list(urls_set)
    
    return unique_urls_list


In [ ]:
# ## Test-Block !DO NOT EXECUTE
# unique_hrefs = unique_urls(all_hrefs_list)
# print(len(unique_hrefs))

# # Test for function unique_urls passed

In [4]:
def regex_pattern_urls(pattern, hrefs_list): 
    """
    Uses a regex pattern to extract urls of interest out of a list of urls
    pattern: regex pattern used for extracting urls from a list of urls
    hrefs_list: list of hrefs required to be filtered according to the regex pattern
    returns:urls_of_interest a list of urls filtered according to regex pattern provided
        
    """
    # Filter the list 'all_hrefs' to keep only the company profile page urls

    #import regex library
    import re
    # regex pattern for company profile pages
    # Do not uncomment following line of code
#     pattern = r'https:\/\/\w+\.wir\.ch\/de\/companyProfile\/profile\/[0-9A-F]{32}\/info\/\?promo=false$'

    urls_of_interest = []

    # loop over all_hrefs list extracted above with Selenium to extract company profile urls
    for href in hrefs_list:
        match = re.search(pattern, str(href))
        if match:
            url = (match.group())
            urls_of_interest.append(url)
    return urls_of_interest        

In [ ]:
# ## Test-Block !DO NOT EXECUTE

# pattern = r'https:\/\/\w+\.wir\.ch\/de\/companyProfile\/profile\/[0-9A-F]{32}\/info\/\?promo=false$'
# company_profile_urls =  regex_pattern_urls(pattern, unique_hrefs)
# print(len(company_profile_urls))
# unique_company_profile_urls = unique_urls(company_profile_urls)
# print(len(unique_company_profile_urls)) # should be 200 (slight variation of up to 5% is acceptable due to variation in internet speed)
# print(unique_company_profile_urls[:10]) #must contain the string "info" in them

# # Test for regex_pattern_urls passed

In [5]:
def extract_info_of_interest(url_list, batch_size = 1000, batch_start_index = 0, sleep=10):
    """
    Extracts all the relevant information from the given unique pages using selenium web driver finders and locators
    Args:
    url_list: list of the pages from where info is to be extracted
    batch_size (int): Number of pages to be processed at a time. It is recommended to process large number of pages in batches of 1000
    batch_start_index(int): index of the url_list from where parsing should start
    sleep(int): seconds to wait for page to load before info can be parsed. It overrides WebDriverWait() function
    Returns:
    list of list of extracted data where each list contains data from a single page
    """
    # Dont't uncomment following line of code, it is for troubleshooting
#   from contextlib import suppress

    # prepare the urls batch to be processed
    batch = url_list[batch_start_index:batch_start_index+batch_size+1]

    company_data = [] # initialize a list to contain lists of all companies' info


    count = 0

    for url in batch:
        company_info = [] # initialize a list to contain one company's info


        # driver control block
        driver = webdriver.Chrome(service=service)
        driver.get(url)
        time.sleep(sleep) # We are using sleep to avoid errors of WebDriverWait though it is timewise costly


    # Using suppress context to ignore known exceptions
    # Do not uncomment following line of code as it may lead to execution errors it is for testing
    #     with suppress(Exception): 

        # Get company names

        try:
    #         WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "71-lnk")))

            h3_headings = driver.find_elements(By.CLASS_NAME, "inline-block") # find all elements with class = "inline-block"
            h3_list = []
            for heading in h3_headings: #loop through the list obtained
                h3_list.append(heading.text) # get text of all h3 headings
            company_name = h3_list[0] # company name is the first item in the list
            company_info.append(company_name)   
        except:
            company_info.append('NA')
        finally:
                pass


        # Get company industry

        try:

            uls = driver.find_elements(By.CLASS_NAME, "default")
            list_items = driver.find_elements(By.TAG_NAME,"li")   
            li_list = []
            for li in list_items:
                li_list.append(li.text)
            industry_name = li_list[20] # industry name is the twenty first item in the list
            company_info.append(industry_name)   
        except:
            company_info.append('NA')
        finally:
                pass

        # Get company website if present         
        try:
    #       WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "68-lnk")))
            anchor = driver.find_element(By.ID, "68-lnk")
            website = anchor.get_attribute('href')
            company_info.append(website)
        except:
                company_info.append('NA')
        finally:
                pass

        # Get company telephoone if available
        try:
    #         WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "66-lnk")))
            anchor = driver.find_element(By.ID, "66-lnk")
            company_telephone = anchor.get_attribute('href')
            company_info.append(company_telephone)
        except:
            company_info.append('NA')
        finally:
                pass
        # Get company fax if available
        try:
    #         WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "fax-lnk")))
            anchor = driver.find_element(By.ID, "fax-lnk")
            company_fax_no = anchor.get_attribute('href')
            company_info.append(company_fax_no)
        except:
            company_info.append('NA')
        finally: 
                pass

       # Get company address and address google map link
        try:
    #       WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "6-lnk")))
            anchor = driver.find_element(By.ID, "6-lnk")
            address = anchor.text
            address_map_link = anchor.get_attribute('href')
            company_info.append(address)
            company_info.append(address_map_link)

        except:
                company_info.append('NA')
        finally:
                pass


        company_data.append(company_info)
        company_info = [] # reinitialize company_info
        
        count+=1
        print("count", count)

        driver.quit()
    return company_data


In [ ]:
# ## Test-Block !DO NOT Execute
# company_data_lists = extract_info_of_interest(url_list = unique_company_profile_urls, batch_size = 200, batch_start_index = 0, sleep=10)

In [ ]:
# # Test-Block !Do Not Execute
# print(len(company_data_lists)) # should be equal to batch size i.e. 200
# # test for extract_info_of_interest passed

In [ ]:
# # Test-Block !DO NOT ExECUTE
# print(len(company_data_lists[-1]))
# print(company_data_lists[:5]) 
# # Should be a list of lists
# # Each list should have seven items
# # First item in each list should be 'company name', second: 'industry', third: website, 
# # fourth: 'tel', fifth 'fax', sixth: 'address', seventh: 'address maplink'
# # data not available should be represented by appropriate remarks

# # test for extract_info_of_interest_passed

In [6]:
def extract_emails_from_homepage(info_list, pattern=r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', index=2):
    """Extracts emails from a list of lists containing a referred website address using a regex pattern
    Args:
    info_list: list of lists containing extracted info including website address
    pattern: regex pattern to cover all possible formats of email addresses
    index(int): index number in the lists where website address is located, default=2
    Returns: info_list_with_emails a list with email addresses appended to info_list
    """

    import requests
    import re
    from bs4 import BeautifulSoup

    info_list_with_emails = info_list.copy()  # make a copy to avoid overwriting original list
    all_links = []
    for page in info_list_with_emails:
        try:
            home_page = str(page[index])
            response = requests.get(home_page)

            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                hrefs = [link.get('href') for link in soup.find_all('a')]  # get all links on the homepage
                # search for emails in homepage response text
                emails = re.findall(pattern, response.text)
                if emails:
                    unique_emails = set(emails)
                    unique_emails_list = list(unique_emails)
                    page.append(unique_emails_list[0])
                    all_links.append(unique_emails_list[0])
                else:
                    # search for emails in homepage hrefs
                    for href in hrefs:
                        if href and href.startswith (str('mailto:')):
                            emails = [] 
                            email = href.split(':')[1]
                            emails.append(email)
                            page.append(emails[0])
                            all_links.append(email)
                            break
                        else:
                            pass
                        
            else:
                page.append('bad link')
        except:
            page.append('website not accessible')

    print('No of emails found: ', len(all_links))
   
    return info_list_with_emails


In [8]:
def extract_emails_from_allpages(info_list, pattern=r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', index=2):
    """Extracts emails from a list of lists containing a referred website address using a regex pattern
    Args:
    info_list: list of lists containing extracted info including website address
    pattern: regex pattern to cover all possible formats of email addresses
    index(int): index number in the lists where website address is located, default=2
    Returns: info_list_with_emails a list with email addresses appended to info_list
    """

    import requests
    import re
    from bs4 import BeautifulSoup

    info_list_with_emails = info_list.copy()  # make a copy to avoid overwriting original list
    all_links = []
    
    for page in info_list_with_emails:
        try:
            home_page = str(page[index])
            response = requests.get(home_page)

            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                hrefs = [link.get('href') for link in soup.find_all('a')]  # get all links on the homepage

                # search for emails on the homepage
                emails = re.findall(pattern, response.text)
                if emails:
                    unique_emails = set(emails)
                    unique_emails_list = list(unique_emails)
                    page.append(unique_emails_list[0])
                    all_links.append(unique_emails_list[0])
                else:
                    # search for emails on all links found
                    email_found = False
                    for href in hrefs:
                        if href and href.startswith(str('mailto:')):
                            emails = [] 
                            email = href.split(':')[1]
                            emails.append(email)
                            page.append(emails[0])
                            all_links.append(emails[0])
                            email_found = True
                            break
                        elif 'http' not in href:
                            link = home_page[:-1]+ href
                            link_response = requests.get(link)
                            if link_response.status_code == 200:
                                link_soup = BeautifulSoup(link_response.text, 'html.parser')
                                link_hrefs = [link.get('href') for link in link_soup.find_all('a')]

                                emails = re.findall(pattern, link_response.text)
                                if emails:
                                    unique_emails = set(emails)
                                    unique_emails_list = list(unique_emails)
                                    page.append(unique_emails_list[0])
                                    all_links.append(unique_emails_list[0])
                                    email_found = True
                                    break
                                        
                    if not email_found:
                        page.append("NA")
                        
            else:
                page.append('bad link')
        except:
            page.append('website not accessible')

    print('No of emails found: ', len(all_links), all_links)

   
    return info_list_with_emails


In [ ]:
# ## Test_Bloack !DO NOT EXECUTE
# company_data_lists_with_emails = extract_emails_from_websites(company_data_lists)

In [ ]:
# ## Test_Block !DO NOT EXECUTE
# print(len(company_data_lists_with_emails)) # should be same as company_data_lists i.e. 200
# print(len(company_data_lists_with_emails[-1])) # should be 8
# print(company_data_lists_with_emails[-5:]) #should have 8 items in each list with emails or remarks added

# # Test for extract_emails_from_websites passed

In [ ]:
# ## Test-Block !DO NOT EXECUTE
# # Convert to Dataframe first 100 items
# columns=['company_name', 'industry', 'website', 'telephone', 'fax', 'address', 'address_maplink', 'email']

# df1 = pd.DataFrame(company_data_lists_with_emails[:100], columns=columns)
 

In [ ]:
# ## Test-Block !DO NOT EXECUTE
# df1.info()

In [ ]:
# ## Test_Block !DO NOT EXECUTE
# df1.head()

In [ ]:
# ## Test-Block !DO NOT EXECUTE
# # Convert to DataFrame next 100 items
# columns=['company_name', 'industry', 'website', 'telephone', 'fax', 'address', 'address_maplink', 'email']

# df2 = pd.DataFrame(company_data_lists_with_emails[101:200], columns=columns)


In [ ]:
# ## Test-Block !DO NOT EXECUTE
# df2.info()

In [ ]:
# ## Test_Block !DO NOT EXECUTE
# df2.head()

In [ ]:
# ## Test_Block !DO NOT EXECUTE
# df2.tail()

In [ ]:
# # Test-Block !DO nOT EXECUTE
# # Join both Dataframes
# df = pd.concat([df1, df2])

In [ ]:
## Test-Block DO NOT EXECUTE
# df.info()

In [ ]:
# ## Test_Block !DO NOT EXECUTE
# df.head()==df1.head() # should be same as df1.head()
# # test passed

In [ ]:
# ## Test_Block !DO NOT EXECUTE
# df.tail()==df2.tail() # should be same as df2.tail()
# # test passed

### xxxxxxxxxxxxxxxxxxxxxx END OF CODE BLOCKXXXXXXXXXXXXXXXXXXXXXXXX

## XXXXXXXXXXXXX START OF EXECUTION BLOCXXXXXXXXXXXXXX

### Part-I Extract links to all company profile pages

In [7]:
## all_hrefs already extracted. Uncomment and execute only if extracting all_hrefs again
# # extract all_hrefs from 224 list pages on the site 
# start_time = time.time()
# # url = f"https://wirmarket.wir.ch/de/members/list/?page={i}&resultAmount=25"
# all_hrefs = extract_hrefs( n=224, sleep = 17)

# end_time = time.time()
# execution_time = end_time - start_time
# print("execution_time: ", execution_time)


execution_time:  7432.664510250092


In [8]:
## Test-Block !Uncomment and execute only if extracting all_hrefs again
# print(len(all_hrefs))

104806


In [61]:
# ## all_hrefs.csv already saved. Uncomment and execute only if extracting all_hrefs again
# all_hrefs_dict = {"hrefs": all_hrefs}
# all_hrefs_df = pd.DataFrame(all_hrefs)
# all_hrefs_df.to_csv("all_hrefs.csv", index = False)

In [9]:
# # Duplicates already removed. !Uncomment and execute only if extracting again
# # Remove duplicate or repeated entries from the all_hrefs
# all_hrefs_unique = unique_urls(all_hrefs) 

In [10]:
# # Test-Block !Uncomment and execute only if extracting again
# print(len(all_hrefs_unique))

29124


In [11]:
# ## Already extracted !Uncomment and execute only if extracting again
# # Extract company profile page urls from all unique hrefs
# pattern = r'https:\/\/\w+\.wir\.ch\/de\/companyProfile\/profile\/[0-9A-F]{32}\/info\/\?promo=false$'
# company_profile_urls = regex_pattern_urls(pattern = pattern, hrefs_list = all_hrefs_unique)

In [12]:
# # Test_Block !Uncomment and execute only if extracting again
# print(len(company_profile_urls))

14227


In [13]:
# # Uncomment and execute only if extracting again
# # Remove duplicate or repeated entries from company_profile_urls
# unique_company_profile_urls = unique_urls(company_profile_urls)

#### Checkpoint-1

In [14]:
# ## Test-Block. !Uncomment and execute only if extracting again
# print(len(unique_company_profile_urls)) # should be 14227 (slight variation of up to 5% is acceptable due to variation in internet speed)
# print(unique_company_profile_urls[:10])
# print(unique_company_profile_urls[-10:])

14227
['https://wirmarket.wir.ch/de/companyProfile/profile/3E429FAF14ED2A65E0540010E0244DC9/info/?promo=false', 'https://wirmarket.wir.ch/de/companyProfile/profile/3E429FAF4B042A65E0540010E0244DC9/info/?promo=false', 'https://wirmarket.wir.ch/de/companyProfile/profile/3E429FAF240E2A65E0540010E0244DC9/info/?promo=false', 'https://wirmarket.wir.ch/de/companyProfile/profile/3E429FAF3B6B2A65E0540010E0244DC9/info/?promo=false', 'https://wirmarket.wir.ch/de/companyProfile/profile/3E429FAF41082A65E0540010E0244DC9/info/?promo=false', 'https://wirmarket.wir.ch/de/companyProfile/profile/3E429FAF12A92A65E0540010E0244DC9/info/?promo=false', 'https://wirmarket.wir.ch/de/companyProfile/profile/3E429FAF1B3B2A65E0540010E0244DC9/info/?promo=false', 'https://wirmarket.wir.ch/de/companyProfile/profile/3E429FAF54932A65E0540010E0244DC9/info/?promo=false', 'https://wirmarket.wir.ch/de/companyProfile/profile/3E429FAF31EB2A65E0540010E0244DC9/info/?promo=false', 'https://wirmarket.wir.ch/de/companyProfile/prof

In [62]:
# # company_profile_url already saved as profile_urls.csv. Uncomment and execute only if extracting again
# # Save as csv
# company_profile_url_dict = {"urls": unique_company_profile_urls}
# company_profile_url_df = pd.DataFrame(company_profile_url_dict)
# company_profile_url_df.to_csv('profile_urls.csv', index = False)

In [10]:
## Comment this code (Ctrl+A followed by Ctrl+/) if extracting hrefs and urls again
# import profile_urls.csv as dataframe
unique_company_profiles_df = pd.read_csv("profile_urls.csv")

In [13]:
## Comment this code (Ctrl+A followed by Ctrl+/) if extracting hrefs and urls again
unique_company_profiles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14227 entries, 0 to 14226
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   urls    14227 non-null  object
dtypes: object(1)
memory usage: 111.3+ KB


In [19]:
## Comment this code (Ctrl+A followed by Ctrl+/) if extracting hrefs and urls again
unique_company_profile_urls = unique_company_profiles_df['urls'].tolist()


In [20]:
## Comment this code (Ctrl+A followed by Ctrl+/) if extracting hrefs and urls again
print(len(unique_company_profile_urls))

14227


In [21]:
## Comment this code (Ctrl+A followed by Ctrl+/) if extracting hrefs and urls again
print(unique_company_profile_urls[-5:])

['https://wirmarket.wir.ch/de/companyProfile/profile/3E429FAF26282A65E0540010E0244DC9/info/?promo=false', 'https://wirmarket.wir.ch/de/companyProfile/profile/3E429FAF3C152A65E0540010E0244DC9/info/?promo=false', 'https://wirmarket.wir.ch/de/companyProfile/profile/58B85543DFA09858E05400144FF95A47/info/?promo=false', 'https://wirmarket.wir.ch/de/companyProfile/profile/3E429FAF27E62A65E0540010E0244DC9/info/?promo=false', 'https://wirmarket.wir.ch/de/companyProfile/profile/3E429FAF55272A65E0540010E0244DC9/info/?promo=false']


### xxxxxxxxxxxxxxxxxxx END OF PART-1 xxxxxxxxxxxxxxxxxxxxxxxxxxx

## Part-2 Batch Processing

### Batch-1, batch_size = 1000, batch_start_index = 0

#### !Cell is executed. Uncomment (Ctrl + A followed by Ctrl + /) and execute (Shift + Enter) again only if you want to run the batch again.

In [17]:
# # Extract info of interst
# start_time = time.time()
# batch1_initial = extract_info_of_interest(unique_company_profile_urls, batch_size = 1000, batch_start_index = 0, sleep=10)
# end_time = time.time()
# execution_time = end_time - start_time
# print('execution time: ', execution_time)

execution time:  18019.821753025055


### checkpoint-1

In [20]:
# # Test-Block ! Execute only if you run the batch again
# print(len(batch1_initial)) # Should be equal to batch_size + 1
# print(len(batch1_initial[-1])) # Should be equal to 7

1001
7


In [21]:
## Test_Block ! Execute only if you run the batch again
# print(batch1_initial[:3])

[['Ringler + Strahm Storenbau AG', 'Sonstige Bauinstallation', 'http://www.ringler-strahm.ch/', 'tel:033 345 22 55', 'fax:033 345 55 19', 'Uetendorfstrasse 20\n3634 Thierachern', 'https://www.google.com/maps/place/Uetendorfstrasse%2020+3634+Thierachern'], ['Grob & Partner Architektur AG', 'Architekturbüros', 'http://www.grobarchitektur.ch/', 'tel:081 720 02 00', 'fax:081 720 02 05', 'Bahnhofstrasse 3\n7320 Sargans', 'https://www.google.com/maps/place/Bahnhofstrasse%20%203+7320+Sargans'], ['Lauclair AG', 'Schreinerarbeiten im Innenausbau', 'http://www.lauclair.ch/', 'tel:031 879 01 69', 'fax:031 879 20 69', 'Lyssstrasse 27\n3054 Schüpfen', 'https://www.google.com/maps/place/Lyssstrasse%2027+3054+Sch%C3%BCpfen']]


### End of checkpoint-1

In [9]:
batch1df = pd.read_csv('batch1.csv')

In [10]:
batch1df_initial = batch1df.drop(['email'], axis = 1)


In [11]:
batch1df_initial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     1001 non-null   object
 1   industry         1001 non-null   object
 2   website          578 non-null    object
 3   telephone        864 non-null    object
 4   fax              572 non-null    object
 5   address          1001 non-null   object
 6   address_maplink  1001 non-null   object
dtypes: object(7)
memory usage: 54.9+ KB


In [12]:
batch1_initial = batch1df_initial.values.tolist()
print(batch1_initial[:3])

[['Ringler + Strahm Storenbau AG', 'Sonstige Bauinstallation', 'http://www.ringler-strahm.ch/', 'tel:033 345 22 55', 'fax:033 345 55 19', 'Uetendorfstrasse 20\n3634 Thierachern', 'https://www.google.com/maps/place/Uetendorfstrasse%2020+3634+Thierachern'], ['Grob & Partner Architektur AG', 'Architekturbüros', 'http://www.grobarchitektur.ch/', 'tel:081 720 02 00', 'fax:081 720 02 05', 'Bahnhofstrasse 3\n7320 Sargans', 'https://www.google.com/maps/place/Bahnhofstrasse%20%203+7320+Sargans'], ['Lauclair AG', 'Schreinerarbeiten im Innenausbau', 'http://www.lauclair.ch/', 'tel:031 879 01 69', 'fax:031 879 20 69', 'Lyssstrasse 27\n3054 Schüpfen', 'https://www.google.com/maps/place/Lyssstrasse%2027+3054+Sch%C3%BCpfen']]


#### Following Function Extracts emails from All Pages of a Site (Takes A Lot of Time: 7-15 sec per page)


In [13]:
# Extract emails
start_time = time.time()
pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
batch1 = extract_emails_from_allpages(batch1_initial, pattern = pattern , index=2)
end_time = time.time()
execution_time = end_time - start_time
print('execution time: ', execution_time)

C:\Users\Admin\miniconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


No of emails found:  336 ['info@ringler-strahm.ch', '605a7baede844d278b89dc95ae0a9123@sentry-next.wixpress.com', 'a95fca1f1eb9fe8g2c9ead0cd9931e2a@2x.jpg', 'info@clubgoldwand.ch', 'payments@2x.png', 'parto@parto.ch', 'info@hunkeler-hediger.ch', 'office@walz-backkunst.ch', 'info@meer.ch', 'peter.flury@garageflury.ch', 'appenzell@hirn.ch', 'info@storit.ch', 'info@kimibau.ch', 'info@rexmetall.ch', 'shop@osgmbh.ch', 'info@mot-goals.ch', 'kontakt@storen-gerber.ch', 'info@spritzwerk-mosberger.ch', 'info@meisterdrogerie.ch', 'alexmuster@example.com', 'info@burgstaller-zaun.ch', 'info@sbh-ag.ch', 'typo3@iresults.li', 'info@dotcon.ch', 'info@panetteriamagliasina.ch', 'info@maxschuepbachag.ch', 'info@jonabau.ch', 'info@felixtransport.ch', '605a7baede844d278b89dc95ae0a9123@sentry-next.wixpress.com', '605a7baede844d278b89dc95ae0a9123@sentry-next.wixpress.com', 'info@felsag.ch', 'info@dussbauservice.ch', 'info@taconla.ch', 'info@widler-ag.ch', 'daniel@abcprint.ch', 'info@diethelm-bodenbelaege.ch', 

### checkpoint-2

In [14]:
print(len(batch1)) # should be same as batch_size i.e. 1001
print(len(batch1[-1])) # should be 8
print(batch1[-5:]) #should have 8 items in each list with emails or remarks added


1001
8
[['STB Holzbau GmbH', 'Einbau von Fenster, Türen und Innenausbau, Einbauküchen, Einbaumöbel', nan, 'tel:055 444 31 68', 'fax:055 444 31 68', 'Haslenstrasse 28b\n8862 Schübelbach', 'https://www.google.com/maps/place/Haslenstrasse%2028b+8862+Sch%C3%BCbelbach', 'website not accessible'], ['Beauty Solar Sonnenland GmbH', 'Saunas, Solarien', nan, 'tel:041 280 15 15', 'fax:041 631 08 07', 'Schulhausstrasse 4\n6052 Hergiswil NW', 'https://www.google.com/maps/place/Schulhausstrasse%204+6052+Hergiswil%20NW', 'website not accessible'], ['Elektro-Brizzi AG', 'Elektroinstallation', nan, 'tel:052 386 22 22', 'fax:052 386 22 93', 'Heinrich Gujer-Strasse 5\n8494 Bauma', 'https://www.google.com/maps/place/Heinrich%20Gujer-Strasse%205+8494+Bauma', 'website not accessible'], ['Keller Wärme & Wasser AG', 'Sanitär- und Heizungsinstallation', nan, 'tel:032 373 51 53', 'fax:032 373 27 71', 'Erlenstrasse 15\n2555 Brügg BE', 'https://www.google.com/maps/place/Erlenstrasse%2015+2555+Br%C3%BCgg%20BE', 'w

### End of checkpoint-2

In [15]:
# Columns to be used for all batches
columns=['company_name', 'industry', 'website', 'telephone', 'fax', 'address', 'address_maplink', 'email_all']


In [16]:
# # Convert to Dataframe batch1
df1 = pd.DataFrame(batch1, columns=columns)


### Checkpoint-3

In [17]:
# # Test-Block ! Execute only if you run the batch again
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     1001 non-null   object
 1   industry         1001 non-null   object
 2   website          578 non-null    object
 3   telephone        864 non-null    object
 4   fax              572 non-null    object
 5   address          1001 non-null   object
 6   address_maplink  1001 non-null   object
 7   email_all        1001 non-null   object
dtypes: object(8)
memory usage: 62.7+ KB


In [18]:
df1.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
0,Ringler + Strahm Storenbau AG,Sonstige Bauinstallation,http://www.ringler-strahm.ch/,tel:033 345 22 55,fax:033 345 55 19,Uetendorfstrasse 20\n3634 Thierachern,https://www.google.com/maps/place/Uetendorfstr...,info@ringler-strahm.ch
1,Grob & Partner Architektur AG,Architekturbüros,http://www.grobarchitektur.ch/,tel:081 720 02 00,fax:081 720 02 05,Bahnhofstrasse 3\n7320 Sargans,https://www.google.com/maps/place/Bahnhofstras...,605a7baede844d278b89dc95ae0a9123@sentry-next.w...
2,Lauclair AG,Schreinerarbeiten im Innenausbau,http://www.lauclair.ch/,tel:031 879 01 69,fax:031 879 20 69,Lyssstrasse 27\n3054 Schüpfen,https://www.google.com/maps/place/Lyssstrasse%...,a95fca1f1eb9fe8g2c9ead0cd9931e2a@2x.jpg
3,Club Goldwand,"Kauf und Verkauf von eigenen Grundstücken, Geb...",http://www.clubgoldwand.ch/,tel:056 282 30 50,NaN,Landstrasse 6\n5415 Nussbaumen AG,https://www.google.com/maps/place/Landstrasse%...,info@clubgoldwand.ch
4,NICOLE DIEM Horgen,Detailhandel mit Brillen und anderen Sehhilfen,http://www.nicolediem.ch/,tel:044 770 10 40,NaN,Dorfplatz 3\n8810 Horgen,https://www.google.com/maps/place/Dorfplatz%20...,payments@2x.png


### End of Checkpoint-3

In [19]:
# Save as csv file
df1.to_csv('batch1_all.csv', index = False)

### xxxxxxxxxxxxxxx Push to Github xxxxxxxxxxxxxxxxxxxxxxxxx

### Batch-2, batch_size = 1000, batch_start_index = 1001

#### !Cell is executed. Uncomment (Ctrl + A followed by Ctrl + /) and execute (Shift + Enter) again only if you want to run the batch again.

In [41]:
# # Extract info of interst
# start_time = time.time()
# batch2_initial = extract_info_of_interest(unique_company_profile_urls, batch_size = 1000, batch_start_index = 1001, sleep=10)
# end_time = time.time()
# execution_time = end_time - start_time
# print('execution time: ', execution_time)

execution time:  19495.211315393448


### checkpoint-1

In [42]:
# ## Test_Block ! Execute only if you run the batch again
# print(len(batch2_initial))
# print(len(batch2_initial[-1]))

1001
7


In [43]:
# ## Test_Block ! Execute only if you run the batch again
# print(batch2_initial[:3])

[['Modehaus Peter', 'Detailhandel mit Herrenbekleidung', 'NA', 'tel:041 970 17 57', 'fax:041 970 07 57', 'Hauptgasse 36\n6130 Willisau', 'https://www.google.com/maps/place/Hauptgasse%2036+6130+Willisau'], ['VB ORGANISATION Sàrl', 'Offsetdruck', 'http://www.imprimerievb.ch/', 'tel:079 205 44 53', 'NA', 'chemin St-Hubert 32\n1950 Sion', 'https://www.google.com/maps/place/chemin%20St-Hubert%2032+1950+Sion'], ['Heinz Stucki', 'Anbau von Getreide (ohne Reis), Hülsenfrüchten und Ölsaaten', 'NA', 'tel:031 781 05 03', 'fax:031 781 05 03', 'Hubmatt\n3116 Noflen BE', 'https://www.google.com/maps/place/Hubmatt+3116+Noflen%20BE']]


### End of Checkpoint-1

In [20]:
batch2df = pd.read_csv('batch2.csv')

In [21]:
batch2df_initial = batch2df.drop(['email'], axis = 1)

In [22]:
batch2df_initial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     987 non-null    object
 1   industry         988 non-null    object
 2   website          578 non-null    object
 3   telephone        867 non-null    object
 4   fax              559 non-null    object
 5   address          987 non-null    object
 6   address_maplink  1001 non-null   object
dtypes: object(7)
memory usage: 54.9+ KB


In [23]:
batch2_initial = batch2df_initial.values.tolist()
print(batch2_initial[:3])

[['Modehaus Peter', 'Detailhandel mit Herrenbekleidung', nan, 'tel:041 970 17 57', 'fax:041 970 07 57', 'Hauptgasse 36\n6130 Willisau', 'https://www.google.com/maps/place/Hauptgasse%2036+6130+Willisau'], ['VB ORGANISATION Sàrl', 'Offsetdruck', 'http://www.imprimerievb.ch/', 'tel:079 205 44 53', nan, 'chemin St-Hubert 32\n1950 Sion', 'https://www.google.com/maps/place/chemin%20St-Hubert%2032+1950+Sion'], ['Heinz Stucki', 'Anbau von Getreide (ohne Reis), Hülsenfrüchten und Ölsaaten', nan, 'tel:031 781 05 03', 'fax:031 781 05 03', 'Hubmatt\n3116 Noflen BE', 'https://www.google.com/maps/place/Hubmatt+3116+Noflen%20BE']]


#### Following Function Extracts emails from All Pages of a Site (Takes A Lot of Time: 15 sec per page)


In [24]:
# Extract emails
start_time = time.time()
pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
batch2 = extract_emails_from_allpages(batch2_initial, pattern = pattern , index=2)
end_time = time.time()
execution_time = end_time - start_time
print('execution time: ', execution_time)#### Following Function Extracts emails from All Pages of a Site (Takes A Lot of Time: 15 sec per page)
#### Entire Batch can take upto 5 hours - Uncomment (Ctrl + A followed by Ctrl + / will uncomment the code.
#### Execute(Shift + Enter) only if needed

C:\Users\Admin\miniconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


No of emails found:  347 ['605a7baede844d278b89dc95ae0a9123@sentry-next.wixpress.com', 'kontakt@heizplan.ch', 'info@hoppler-gmbh.ch', 'kontakt@gabrielblumen.ch', '605a7baede844d278b89dc95ae0a9123@sentry-next.wixpress.com', 'info@energies-score.ch', 'info@zubergebrag.ch', 'info@andresdruck.ch', 'anfrage@orderpoint.ch', 'info@fruehimmo.ch', 'info@perfect.ch', 'marie.mustermann@musterag.ch', '605a7baede844d278b89dc95ae0a9123@sentry-next.wixpress.com', '605a7baede844d278b89dc95ae0a9123@sentry-next.wixpress.com', 'info@plastic.ch', 'omp@bluewin.ch', 'info@kottmannherrenmode.ch', 'silvano@canu-bau.ch', 'info@nortec.ch', 'info@schallergroup.ch', 'info.steiner@bluewin.ch', 'info@werbeluond.ch', 'info@blaettler-holzbau.ch', 'harry.huerlimann@print-x.ch', 'besag@besag.ch', 'info@belmediaverlag.com', 'info@pingag.ch', 'bf-logo@1x.png', '605a7baede844d278b89dc95ae0a9123@sentry-next.wixpress.com', 'f.stoll@bnag.ch', 'info@ehb-elektro.ch', '605a7baede844d278b89dc95ae0a9123@sentry-next.wixpress.com',

### Checkpoint-2

In [26]:
print(len(batch2)) # should be same as batch_size i.e. 1000
print(len(batch2[-1])) # should be 8
print(batch2[-5:]) #should have 8 items in each list with emails or remarks added

1001
8
[['MBA AG', 'Grosshandel mit Bergwerks-, Bau- und Baustoffmaschinen', 'http://www.mba-baumaschinen.ch/', 'tel:044 838 61 11', nan, 'Zürichstrasse 50\n8303 Bassersdorf', 'https://www.google.com/maps/place/Z%C3%BCrichstrasse%2050+8303+Bassersdorf', 'info@mba-maschinen.com'], ['Mächler GU AG', 'Garten- und Landschaftsbau sowie Erbringung von sonstigen gärtnerischen Dienstleistungen', 'http://www.maechler-gu.ch/', 'tel:055 451 11 31', 'fax:055 451 11 34', 'St.Gallerstrasse 58\n8853 Lachen SZ', 'https://www.google.com/maps/place/St.Gallerstrasse%2058+8853+Lachen%20SZ', 'info@maechler-gu.ch'], ['Garasch 106 AG', 'lnstandhaltung und Reparatur von Automobilen', 'https://garasch106.ch/', 'tel:041 624 40 24', nan, 'Stanserstrasse 106\n6373 Ennetbürgen', 'https://www.google.com/maps/place/Stanserstrasse%20106+6373+Ennetb%C3%BCrgen', 'auto@garasch106.ch'], ['Bumbachsäge AG', 'Sägewerke', 'http://www.bumbachsaege.ch/', 'tel:034 493 35 57', 'fax:034 493 31 84', 'Bumbachsäge 172\n6197 Schangna

### End of Checkpoint-2

In [27]:
# # Convert to Dataframe batch1
df2 = pd.DataFrame(batch2, columns=columns)


### Checkpoint 3

In [28]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     987 non-null    object
 1   industry         988 non-null    object
 2   website          578 non-null    object
 3   telephone        867 non-null    object
 4   fax              559 non-null    object
 5   address          987 non-null    object
 6   address_maplink  1001 non-null   object
 7   email_all        1001 non-null   object
dtypes: object(8)
memory usage: 62.7+ KB


In [29]:
df2.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
0,Modehaus Peter,Detailhandel mit Herrenbekleidung,NaN,tel:041 970 17 57,fax:041 970 07 57,Hauptgasse 36\n6130 Willisau,https://www.google.com/maps/place/Hauptgasse%2...,website not accessible
1,VB ORGANISATION Sàrl,Offsetdruck,http://www.imprimerievb.ch/,tel:079 205 44 53,NaN,chemin St-Hubert 32\n1950 Sion,https://www.google.com/maps/place/chemin%20St-...,605a7baede844d278b89dc95ae0a9123@sentry-next.w...
2,Heinz Stucki,"Anbau von Getreide (ohne Reis), Hülsenfrüchten...",NaN,tel:031 781 05 03,fax:031 781 05 03,Hubmatt\n3116 Noflen BE,https://www.google.com/maps/place/Hubmatt+3116...,website not accessible
3,Heizplan HPA AG,Sanitär- und Heizungsinstallation,http://www.heizplan.ch/,tel:081 750 34 50,fax:081 750 34 59,Karmaad 36\n9473 Gams,https://www.google.com/maps/place/Karmaad%2036...,kontakt@heizplan.ch
4,Hoppler Tiefbohrungen GmbH,Test- und Suchbohrung,http://www.hoppler-gmbh.ch/,tel:056 634 40 46,fax:056 634 40 75,Allmendstrasse 1\n5621 Zufikon,https://www.google.com/maps/place/Allmendstras...,info@hoppler-gmbh.ch


### End of Checkpoint-3

In [30]:
# save as csv
df2.to_csv('batch2_all.csv', index = False)

In [31]:
# Merge the two dataframes
df = pd.concat([df1, df2])

### Checkpoint-4

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2002 entries, 0 to 1000
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     1988 non-null   object
 1   industry         1989 non-null   object
 2   website          1156 non-null   object
 3   telephone        1731 non-null   object
 4   fax              1131 non-null   object
 5   address          1988 non-null   object
 6   address_maplink  2002 non-null   object
 7   email_all        2002 non-null   object
dtypes: object(8)
memory usage: 140.8+ KB


In [33]:
df.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
0,Ringler + Strahm Storenbau AG,Sonstige Bauinstallation,http://www.ringler-strahm.ch/,tel:033 345 22 55,fax:033 345 55 19,Uetendorfstrasse 20\n3634 Thierachern,https://www.google.com/maps/place/Uetendorfstr...,info@ringler-strahm.ch
1,Grob & Partner Architektur AG,Architekturbüros,http://www.grobarchitektur.ch/,tel:081 720 02 00,fax:081 720 02 05,Bahnhofstrasse 3\n7320 Sargans,https://www.google.com/maps/place/Bahnhofstras...,605a7baede844d278b89dc95ae0a9123@sentry-next.w...
2,Lauclair AG,Schreinerarbeiten im Innenausbau,http://www.lauclair.ch/,tel:031 879 01 69,fax:031 879 20 69,Lyssstrasse 27\n3054 Schüpfen,https://www.google.com/maps/place/Lyssstrasse%...,a95fca1f1eb9fe8g2c9ead0cd9931e2a@2x.jpg
3,Club Goldwand,"Kauf und Verkauf von eigenen Grundstücken, Geb...",http://www.clubgoldwand.ch/,tel:056 282 30 50,NaN,Landstrasse 6\n5415 Nussbaumen AG,https://www.google.com/maps/place/Landstrasse%...,info@clubgoldwand.ch
4,NICOLE DIEM Horgen,Detailhandel mit Brillen und anderen Sehhilfen,http://www.nicolediem.ch/,tel:044 770 10 40,NaN,Dorfplatz 3\n8810 Horgen,https://www.google.com/maps/place/Dorfplatz%20...,payments@2x.png


In [34]:
df.tail()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
996,MBA AG,"Grosshandel mit Bergwerks-, Bau- und Baustoffm...",http://www.mba-baumaschinen.ch/,tel:044 838 61 11,NaN,Zürichstrasse 50\n8303 Bassersdorf,https://www.google.com/maps/place/Z%C3%BCrichs...,info@mba-maschinen.com
997,Mächler GU AG,Garten- und Landschaftsbau sowie Erbringung vo...,http://www.maechler-gu.ch/,tel:055 451 11 31,fax:055 451 11 34,St.Gallerstrasse 58\n8853 Lachen SZ,https://www.google.com/maps/place/St.Gallerstr...,info@maechler-gu.ch
998,Garasch 106 AG,lnstandhaltung und Reparatur von Automobilen,https://garasch106.ch/,tel:041 624 40 24,NaN,Stanserstrasse 106\n6373 Ennetbürgen,https://www.google.com/maps/place/Stanserstras...,auto@garasch106.ch
999,Bumbachsäge AG,Sägewerke,http://www.bumbachsaege.ch/,tel:034 493 35 57,fax:034 493 31 84,Bumbachsäge 172\n6197 Schangnau,https://www.google.com/maps/place/Bumbachs%C3%...,info@bumbachsaege.ch
1000,MEUBLES DESCARTES SA,Inserate,http://www.decarte.ch/,tel:027 743 43 43,fax:027 743 43 44,Route du Léman 33\n1907 Saxon,https://www.google.com/maps/place/Route%20du%2...,NA


In [35]:
df.head() == df1.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
0,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True
3,True,True,True,True,False,True,True,True
4,True,True,True,True,False,True,True,True


In [36]:
df.tail() == df2.tail()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
996,True,True,True,True,False,True,True,True
997,True,True,True,True,True,True,True,True
998,True,True,True,True,False,True,True,True
999,True,True,True,True,True,True,True,True
1000,True,True,True,True,True,True,True,True


### End of Checkpoint-4

### xxxxxxxxxxxx Push to Github xxxxxxxxxxxxxx

### Batch-3, batch_size = 1000, batch_start_index = 2001


#### !Cell is executed. Uncomment (Ctrl + A followed by Ctrl + /) and execute (Shift + Enter) again only if you want to run the batch again.

In [58]:
# # Extract info of interst
# start_time = time.time()
# batch3_initial = extract_info_of_interest(unique_company_profile_urls, batch_size = 1000, batch_start_index = 2001, sleep=5)
# end_time = time.time()
# execution_time = end_time - start_time
# print('execution time: ', execution_time)

execution time:  14860.21987771988


### checkpoint-1

In [63]:
# # Test-Block ! Execute only if running the batch again
# print(len(batch3_initial))
# print(len(batch3_initial[-1]))

1001
7


In [64]:
# # Test-Block ! Execute only if running the batch again
# print(batch3_initial[:3])

[['MEUBLES DESCARTES SA', 'Inserate', 'http://www.decarte.ch/', 'tel:027 743 43 43', 'fax:027 743 43 44', 'Route du Léman 33\n1907 Saxon', 'https://www.google.com/maps/place/Route%20du%20L%C3%A9man%2033+1907+Saxon'], ['Eticolle Schoechli SA', 'Sonstiges Drucken a. n. g.', 'http://www.eticolle.ch/', 'tel:027 452 25 26', 'NA', 'Techno-pôle 2\n3960 Sierre', 'https://www.google.com/maps/place/Techno-p%C3%B4le%202+3960+Sierre'], ['Ettima AG', 'Grosshandel mit Werkzeugmaschinen', 'http://www.ettima.ch/', 'tel:031 819 56 26', 'fax:031 819 56 63', 'Bernstrasse 25\n3125 Toffen', 'https://www.google.com/maps/place/Bernstrasse%2025+3125+Toffen']]


### End of Checkpoint-1

In [37]:
batch3df = pd.read_csv('batch3.csv')

In [38]:
batch3df_initial = batch3df.drop(['email'], axis = 1)

In [39]:
batch3df_initial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     986 non-null    object
 1   industry         985 non-null    object
 2   website          561 non-null    object
 3   telephone        880 non-null    object
 4   fax              548 non-null    object
 5   address          986 non-null    object
 6   address_maplink  1001 non-null   object
dtypes: object(7)
memory usage: 54.9+ KB


In [41]:
batch3_initial = batch3df_initial.values.tolist()
print(batch3_initial[:3])

[['MEUBLES DESCARTES SA', 'Inserate', 'http://www.decarte.ch/', 'tel:027 743 43 43', 'fax:027 743 43 44', 'Route du Léman 33\n1907 Saxon', 'https://www.google.com/maps/place/Route%20du%20L%C3%A9man%2033+1907+Saxon'], ['Eticolle Schoechli SA', 'Sonstiges Drucken a. n. g.', 'http://www.eticolle.ch/', 'tel:027 452 25 26', nan, 'Techno-pôle 2\n3960 Sierre', 'https://www.google.com/maps/place/Techno-p%C3%B4le%202+3960+Sierre'], ['Ettima AG', 'Grosshandel mit Werkzeugmaschinen', 'http://www.ettima.ch/', 'tel:031 819 56 26', 'fax:031 819 56 63', 'Bernstrasse 25\n3125 Toffen', 'https://www.google.com/maps/place/Bernstrasse%2025+3125+Toffen']]


### End of Checkpoint-1

#### Following Function Extracts emails from All Pages of a Site (Takes A Lot of Time: 7-15 sec per page)


In [42]:
# Extract emails
start_time = time.time()
pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
batch3 = extract_emails_from_allpages(batch3_initial, pattern = pattern , index=2)
end_time = time.time()
execution_time = end_time - start_time
print('execution time: ', execution_time)

C:\Users\Admin\miniconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


No of emails found:  333 ['info@eticolle.ch', '605a7baede844d278b89dc95ae0a9123@sentry-next.wixpress.com', 'info@kerteszkabel.ch', 'info@elektrohunziker.ch', 'info@company.com', 'druckerei@bucherdruck.ch', '605a7baede844d278b89dc95ae0a9123@sentry-next.wixpress.com', 'info@tstor.ch', 'info@baubox-partner.ch', 'info@rudolfbeyeler.ch', 'info@swiss-photovoltaik.ch', 'info@wiedmer.ch', 'info@zimmermann-spiez.ch', '605a7baede844d278b89dc95ae0a9123@sentry-next.wixpress.com', 'info@na-vertrieb.ch', 'info@brandschutzschweizag.ch', 'info@adgenerale.ch', 'info@memoform.ch', 'solothurn@gommobil.ch', 'info@harisport.ch', 'info@holzbau-ledergerber.ch', 'r.chappuis@cas-gruppe.ch', 'info@cantex.ch', 'f36cc48fb72b4d298c835f2793cf3b84@sentry-next.wixpress.com', 'info@liem.ch', 'info@schiess-wohntextil.ch', 'bau@bezzola-denoth.ch', '605a7baede844d278b89dc95ae0a9123@sentry-next.wixpress.com', 'info@geruestbau-graf.ch', 'info@oekofen.ch', 'info@bauen-mit-glas.ch', 'info@pneuhaus-wegmann.ch', 'info@netprom.

### Checkpoint-2

In [43]:
print(len(batch3)) # should be same as batch_size i.e. 1000
print(len(batch3[-1])) # should be 8
print(batch3[-5:]) #should have 8 items in each list with emails or remarks added

1001
8
[['Hirt Schreinerei GmbH', 'Schreinerarbeiten im Innenausbau', 'http://www.hirt-schreinerei.ch/', 'tel:061 901 55 88', 'fax:061 901 83 04', 'Bächliackerstrasse 4\n4402 Frenkendorf', 'https://www.google.com/maps/place/B%C3%A4chliackerstrasse%204+4402+Frenkendorf', 'bad link'], ['René Bührer AG', 'Detailhandel mit Vorhängen, Teppichen, Fussbodenbelägen und Tapeten', nan, 'tel:052 672 16 30', 'fax:052 672 16 40', 'Rheinfallstrasse 7\n8212 Neuhausen am Rheinfall', 'https://www.google.com/maps/place/Rheinfallstrasse%207+8212+Neuhausen%20am%20Rheinfall', 'website not accessible'], ['EFOS Flugschule GmbH', 'Fahr- und Flugschulen', 'http://www.efos.ch/', 'tel:044 861 07 04', nan, 'Flughafenstrasse 14\n8302 Kloten', 'https://www.google.com/maps/place/Flughafenstrasse%2014+8302+Kloten', '20ws@efos.ch'], ['Hess Druck AG', 'Offsetdruck', 'http://www.hessdruck.ch/', 'tel:071 658 61 80', nan, 'Schützlerweg 3\n8560 Märstetten', 'https://www.google.com/maps/place/Sch%C3%BCtzlerweg%203+8560+M%C3

### End of Checkpoint-2

In [44]:
# # Convert to Dataframe batch3
df3 = pd.DataFrame(batch3, columns=columns)


### Checkpoint 3

In [45]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     986 non-null    object
 1   industry         985 non-null    object
 2   website          561 non-null    object
 3   telephone        880 non-null    object
 4   fax              548 non-null    object
 5   address          986 non-null    object
 6   address_maplink  1001 non-null   object
 7   email_all        1001 non-null   object
dtypes: object(8)
memory usage: 62.7+ KB


In [46]:
df3.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
0,MEUBLES DESCARTES SA,Inserate,http://www.decarte.ch/,tel:027 743 43 43,fax:027 743 43 44,Route du Léman 33\n1907 Saxon,https://www.google.com/maps/place/Route%20du%2...,NA
1,Eticolle Schoechli SA,Sonstiges Drucken a. n. g.,http://www.eticolle.ch/,tel:027 452 25 26,NaN,Techno-pôle 2\n3960 Sierre,https://www.google.com/maps/place/Techno-p%C3%...,info@eticolle.ch
2,Ettima AG,Grosshandel mit Werkzeugmaschinen,http://www.ettima.ch/,tel:031 819 56 26,fax:031 819 56 63,Bernstrasse 25\n3125 Toffen,https://www.google.com/maps/place/Bernstrasse%...,website not accessible
3,Alushi AG,Spezielle Reinigung von Gebäuden und Reinigung...,http://www.alushi.ch/,tel:052 376 16 80,fax:052 376 16 87,Frauenfelderstrasse 66\n9548 Matzingen,https://www.google.com/maps/place/Frauenfelder...,website not accessible
4,Studer Optik,Detailhandel mit Brillen und anderen Sehhilfen,http://www.studeroptik.ch/,tel:032 392 25 15,fax:032 392 28 89,Stadtplatz 60\n3270 Aarberg,https://www.google.com/maps/place/Stadtplatz%2...,605a7baede844d278b89dc95ae0a9123@sentry-next.w...


### End of Checkpoint-3

In [47]:
#save as csv
df3.to_csv('batch3_all.csv', index = False)

In [48]:
# Merge the two dataframes
df = pd.concat([df, df3])

### Checkpoint-4

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3003 entries, 0 to 1000
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     2974 non-null   object
 1   industry         2974 non-null   object
 2   website          1717 non-null   object
 3   telephone        2611 non-null   object
 4   fax              1679 non-null   object
 5   address          2974 non-null   object
 6   address_maplink  3003 non-null   object
 7   email_all        3003 non-null   object
dtypes: object(8)
memory usage: 211.1+ KB


In [50]:
df.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
0,Ringler + Strahm Storenbau AG,Sonstige Bauinstallation,http://www.ringler-strahm.ch/,tel:033 345 22 55,fax:033 345 55 19,Uetendorfstrasse 20\n3634 Thierachern,https://www.google.com/maps/place/Uetendorfstr...,info@ringler-strahm.ch
1,Grob & Partner Architektur AG,Architekturbüros,http://www.grobarchitektur.ch/,tel:081 720 02 00,fax:081 720 02 05,Bahnhofstrasse 3\n7320 Sargans,https://www.google.com/maps/place/Bahnhofstras...,605a7baede844d278b89dc95ae0a9123@sentry-next.w...
2,Lauclair AG,Schreinerarbeiten im Innenausbau,http://www.lauclair.ch/,tel:031 879 01 69,fax:031 879 20 69,Lyssstrasse 27\n3054 Schüpfen,https://www.google.com/maps/place/Lyssstrasse%...,a95fca1f1eb9fe8g2c9ead0cd9931e2a@2x.jpg
3,Club Goldwand,"Kauf und Verkauf von eigenen Grundstücken, Geb...",http://www.clubgoldwand.ch/,tel:056 282 30 50,NaN,Landstrasse 6\n5415 Nussbaumen AG,https://www.google.com/maps/place/Landstrasse%...,info@clubgoldwand.ch
4,NICOLE DIEM Horgen,Detailhandel mit Brillen und anderen Sehhilfen,http://www.nicolediem.ch/,tel:044 770 10 40,NaN,Dorfplatz 3\n8810 Horgen,https://www.google.com/maps/place/Dorfplatz%20...,payments@2x.png


In [51]:
df.tail()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
996,Hirt Schreinerei GmbH,Schreinerarbeiten im Innenausbau,http://www.hirt-schreinerei.ch/,tel:061 901 55 88,fax:061 901 83 04,Bächliackerstrasse 4\n4402 Frenkendorf,https://www.google.com/maps/place/B%C3%A4chlia...,bad link
997,René Bührer AG,"Detailhandel mit Vorhängen, Teppichen, Fussbod...",NaN,tel:052 672 16 30,fax:052 672 16 40,Rheinfallstrasse 7\n8212 Neuhausen am Rheinfall,https://www.google.com/maps/place/Rheinfallstr...,website not accessible
998,EFOS Flugschule GmbH,Fahr- und Flugschulen,http://www.efos.ch/,tel:044 861 07 04,NaN,Flughafenstrasse 14\n8302 Kloten,https://www.google.com/maps/place/Flughafenstr...,20ws@efos.ch
999,Hess Druck AG,Offsetdruck,http://www.hessdruck.ch/,tel:071 658 61 80,NaN,Schützlerweg 3\n8560 Märstetten,https://www.google.com/maps/place/Sch%C3%BCtzl...,info@hessdruck.ch
1000,Rhomberg Schmuck AG,Detailhandel mit Uhren und Schmuck,http://www.schmuck.ch/,tel:071 310 15 80,NaN,Zürcher-Strasse\n9015 St. Gallen,https://www.google.com/maps/place/Z%C3%BCrcher...,website not accessible


In [52]:
df.head() == df1.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
0,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True
3,True,True,True,True,False,True,True,True
4,True,True,True,True,False,True,True,True


In [53]:
df.tail() == df3.tail()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
996,True,True,True,True,True,True,True,True
997,True,True,False,True,True,True,True,True
998,True,True,True,True,False,True,True,True
999,True,True,True,True,False,True,True,True
1000,True,True,True,True,False,True,True,True


### xxxxxxxxxxxx Push to Github xxxxxxxxxxxxxx

### Batch-4, batch_size = 1000, batch_start_index = 3001


#### !Cell is executed. Uncomment (Ctrl + A followed by Ctrl + /) and execute (Shift + Enter) again only if you want to run the batch again.

In [81]:
# # Extract info of interst
# start_time = time.time()
# batch4_initial = extract_info_of_interest(unique_company_profile_urls, batch_size = 1000, batch_start_index = 3001, sleep=5)
# end_time = time.time()
# execution_time = end_time - start_time
# print('execution time: ', execution_time)

count 1
count 2
count 3
count 4
count 5
count 6
count 7
count 8
count 9
count 10
count 11
count 12
count 13
count 14
count 15
count 16
count 17
count 18
count 19
count 20
count 21
count 22
count 23
count 24
count 25
count 26
count 27
count 28
count 29
count 30
count 31
count 32
count 33
count 34
count 35
count 36
count 37
count 38
count 39
count 40
count 41
count 42
count 43
count 44
count 45
count 46
count 47
count 48
count 49
count 50
count 51
count 52
count 53
count 54
count 55
count 56
count 57
count 58
count 59
count 60
count 61
count 62
count 63
count 64
count 65
count 66
count 67
count 68
count 69
count 70
count 71
count 72
count 73
count 74
count 75
count 76
count 77
count 78
count 79
count 80
count 81
count 82
count 83
count 84
count 85
count 86
count 87
count 88
count 89
count 90
count 91
count 92
count 93
count 94
count 95
count 96
count 97
count 98
count 99
count 100
count 101
count 102
count 103
count 104
count 105
count 106
count 107
count 108
count 109
count 110
count 11

count 832
count 833
count 834
count 835
count 836
count 837
count 838
count 839
count 840
count 841
count 842
count 843
count 844
count 845
count 846
count 847
count 848
count 849
count 850
count 851
count 852
count 853
count 854
count 855
count 856
count 857
count 858
count 859
count 860
count 861
count 862
count 863
count 864
count 865
count 866
count 867
count 868
count 869
count 870
count 871
count 872
count 873
count 874
count 875
count 876
count 877
count 878
count 879
count 880
count 881
count 882
count 883
count 884
count 885
count 886
count 887
count 888
count 889
count 890
count 891
count 892
count 893
count 894
count 895
count 896
count 897
count 898
count 899
count 900
count 901
count 902
count 903
count 904
count 905
count 906
count 907
count 908
count 909
count 910
count 911
count 912
count 913
count 914
count 915
count 916
count 917
count 918
count 919
count 920
count 921
count 922
count 923
count 924
count 925
count 926
count 927
count 928
count 929
count 930
count 931


### checkpoint-1

In [82]:
# # Test-Block ! Uncomment and execute only if batch is run again
# print(len(batch4_initial))
# print(len(batch4_initial[-1]))

1001
7


In [83]:
# # Test-Block ! Uncomment and execute only if batch is run again
# print(batch4_initial[:3])

[['Rhomberg Schmuck AG', 'Detailhandel mit Uhren und Schmuck', 'http://www.schmuck.ch/', 'tel:071 310 15 80', 'NA', 'Zürcher-Strasse\n9015 St. Gallen', 'https://www.google.com/maps/place/Z%C3%BCrcher-Strasse+9015+St.%20Gallen'], ['GanzImmo AG', 'Verwaltung von Grundstücken, Gebäuden und Wohnungen für Dritte', 'http://www.ganzimmo.ch/', 'tel:052 213 56 65', 'fax:052 213 31 74', 'Schaffhauserstrasse 79\n8401 Winterthur', 'https://www.google.com/maps/place/Schaffhauserstrasse%2079+8401+Winterthur'], ['Coiffure Daniel Moll', 'Artikel im Shop', 'http://www.coiffeur-kriens.com/', 'tel:041 310 05 41', 'NA', 'Amlehnstrasse 54\n6010 Kriens', 'https://www.google.com/maps/place/Amlehnstrasse%2054+6010+Kriens']]


### End of Checkpoint-1

In [54]:
batch4df = pd.read_csv('batch4.csv')

In [55]:
batch4df_initial = batch4df.drop(['email'], axis = 1)

In [56]:
batch4df_initial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     1001 non-null   object
 1   industry         1001 non-null   object
 2   website          563 non-null    object
 3   telephone        883 non-null    object
 4   fax              530 non-null    object
 5   address          1001 non-null   object
 6   address_maplink  1001 non-null   object
dtypes: object(7)
memory usage: 54.9+ KB


In [57]:
batch4_initial = batch4df_initial.values.tolist()
print(batch1_initial[:3])

[['Ringler + Strahm Storenbau AG', 'Sonstige Bauinstallation', 'http://www.ringler-strahm.ch/', 'tel:033 345 22 55', 'fax:033 345 55 19', 'Uetendorfstrasse 20\n3634 Thierachern', 'https://www.google.com/maps/place/Uetendorfstrasse%2020+3634+Thierachern', 'info@ringler-strahm.ch'], ['Grob & Partner Architektur AG', 'Architekturbüros', 'http://www.grobarchitektur.ch/', 'tel:081 720 02 00', 'fax:081 720 02 05', 'Bahnhofstrasse 3\n7320 Sargans', 'https://www.google.com/maps/place/Bahnhofstrasse%20%203+7320+Sargans', '605a7baede844d278b89dc95ae0a9123@sentry-next.wixpress.com'], ['Lauclair AG', 'Schreinerarbeiten im Innenausbau', 'http://www.lauclair.ch/', 'tel:031 879 01 69', 'fax:031 879 20 69', 'Lyssstrasse 27\n3054 Schüpfen', 'https://www.google.com/maps/place/Lyssstrasse%2027+3054+Sch%C3%BCpfen', 'a95fca1f1eb9fe8g2c9ead0cd9931e2a@2x.jpg']]


#### Following Function Extracts emails from All Pages of a Site (Takes A Lot of Time: 7-15 sec per page)


In [58]:
# Extract emails
start_time = time.time()
pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
batch4 = extract_emails_from_allpages(batch4_initial, pattern = pattern , index=2)
end_time = time.time()
execution_time = end_time - start_time
print('execution time: ', execution_time)

No of emails found:  350 ['605a7baede844d278b89dc95ae0a9123@sentry-next.wixpress.com', 'cid_image001_png@01D443CD.png', 'info@cristallina.ch', 'kst@konrads.villas', 'pugin@rohrer-marti.ch', 'info@knupp.ch', '605a7baede844d278b89dc95ae0a9123@sentry-next.wixpress.com', 'info@hefti-kuechen.ch', 'info@blumenhaus-viola.ch', 'graphic.noel@bluewin.ch', 'logo_badge_wir-machen-profis@2x.png', 'info@rohrer-gartenbau.ch', 'info@reut-schreinerei.ch', 'info@electro-stettlerthoenen.ch', 'ralf.seelig@somedia.ch', 'info@guntlin.ch', 'gregor.waldburger@sazburgdorf.ch', 'info@km-kuechenstudio.ch', 'info@gnusspur.ch', 'info@liftag.ch', 'johndoe@domain.com', 'info@rosenberger-holzbau.ch', 'kontakt@giroud-olma.ch', 'email@email.ch', 'info@airproduct.ch', 'info@zollinger.ch', 'hans.hofstetter@hofstetter-world.com', 'patrick.vetter@egeli.ch', 'info@hotel-du-lac.ch', '605a7baede844d278b89dc95ae0a9123@sentry-next.wixpress.com', 'dandar@icipit.ch', 'info@krummenacher-sanitaer.ch', 'info@bucherarchitektur.ch', '

### Checkpoint-2

In [59]:
print(len(batch4)) # should be same as batch_size i.e. 1000
print(len(batch4[-1])) # should be 8
print(batch4[-5:]) #should have 8 items in each list with emails or remarks added

1001
8
[['Martin Stoller Transporte', 'Güterbeförderung im Strassenverkehr', nan, 'tel:031 741 24 15', nan, 'Weidstrasse 2\n3184 Wünnewil', 'https://www.google.com/maps/place/Weidstrasse%202+3184+W%C3%BCnnewil', 'website not accessible'], ['Christoph Hasler Schreinerei', 'Schreinerarbeiten im Innenausbau', 'http://www.hasler-schreinerei.ch/', 'tel:071 917 11 55', 'fax:071 917 18 48', 'Käsereistrasse 1\n9555 Tobel', 'https://www.google.com/maps/place/K%C3%A4sereistrasse%201+9555+Tobel', 'info@hasler-schreinerei.ch'], ['MoRailSo AG', 'Grosshandel mit sonstigen Maschinen und Ausrüstungen', 'https://www.ramorail.com/', 'tel:079 463 97 50', nan, 'Rothmatte 7\n6022 Grosswangen', 'https://www.google.com/maps/place/Rothmatte%207+6022+Grosswangen', 'bad link'], ['Elektro Schuler AG', 'Elektroinstallation', nan, 'tel:041 631 03 31', 'fax:041 631 03 32', 'Schulhausstrasse 4\n6052 Hergiswil NW', 'https://www.google.com/maps/place/Schulhausstrasse%204+6052+Hergiswil%20NW', 'website not accessible']

### End of Checkpoint-2

In [60]:
# # Convert to Dataframe batch1
df4 = pd.DataFrame(batch4, columns=columns)


### Checkpoint 3

In [61]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     1001 non-null   object
 1   industry         1001 non-null   object
 2   website          563 non-null    object
 3   telephone        883 non-null    object
 4   fax              530 non-null    object
 5   address          1001 non-null   object
 6   address_maplink  1001 non-null   object
 7   email_all        1001 non-null   object
dtypes: object(8)
memory usage: 62.7+ KB


In [62]:
df4.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
0,Rhomberg Schmuck AG,Detailhandel mit Uhren und Schmuck,http://www.schmuck.ch/,tel:071 310 15 80,NaN,Zürcher-Strasse\n9015 St. Gallen,https://www.google.com/maps/place/Z%C3%BCrcher...,website not accessible
1,GanzImmo AG,"Verwaltung von Grundstücken, Gebäuden und Wohn...",http://www.ganzimmo.ch/,tel:052 213 56 65,fax:052 213 31 74,Schaffhauserstrasse 79\n8401 Winterthur,https://www.google.com/maps/place/Schaffhauser...,website not accessible
2,Coiffure Daniel Moll,Artikel im Shop,http://www.coiffeur-kriens.com/,tel:041 310 05 41,NaN,Amlehnstrasse 54\n6010 Kriens,https://www.google.com/maps/place/Amlehnstrass...,605a7baede844d278b89dc95ae0a9123@sentry-next.w...
3,Rondell AG,"Kauf und Verkauf von eigenen Grundstücken, Geb...",NaN,tel:041 320 53 73,NaN,Weinhalde 12\n6010 Kriens,https://www.google.com/maps/place/Weinhalde%20...,website not accessible
4,Rémy Bühler Malergeschäft Rothrist,Malerei,http://www.maler-buehler.ch/,tel:062 794 44 64,NaN,Pfaffernweg 7\n4852 Rothrist,https://www.google.com/maps/place/Pfaffernweg%...,cid_image001_png@01D443CD.png


### End of Checkpoint-3

In [63]:
#save as csv
df4.to_csv('batch4_all.csv', index = False)

In [64]:
# Merge the two dataframes
df = pd.concat([df, df4])

### Checkpoint-4

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4004 entries, 0 to 1000
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     3975 non-null   object
 1   industry         3975 non-null   object
 2   website          2280 non-null   object
 3   telephone        3494 non-null   object
 4   fax              2209 non-null   object
 5   address          3975 non-null   object
 6   address_maplink  4004 non-null   object
 7   email_all        4004 non-null   object
dtypes: object(8)
memory usage: 281.5+ KB


In [66]:
df.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
0,Ringler + Strahm Storenbau AG,Sonstige Bauinstallation,http://www.ringler-strahm.ch/,tel:033 345 22 55,fax:033 345 55 19,Uetendorfstrasse 20\n3634 Thierachern,https://www.google.com/maps/place/Uetendorfstr...,info@ringler-strahm.ch
1,Grob & Partner Architektur AG,Architekturbüros,http://www.grobarchitektur.ch/,tel:081 720 02 00,fax:081 720 02 05,Bahnhofstrasse 3\n7320 Sargans,https://www.google.com/maps/place/Bahnhofstras...,605a7baede844d278b89dc95ae0a9123@sentry-next.w...
2,Lauclair AG,Schreinerarbeiten im Innenausbau,http://www.lauclair.ch/,tel:031 879 01 69,fax:031 879 20 69,Lyssstrasse 27\n3054 Schüpfen,https://www.google.com/maps/place/Lyssstrasse%...,a95fca1f1eb9fe8g2c9ead0cd9931e2a@2x.jpg
3,Club Goldwand,"Kauf und Verkauf von eigenen Grundstücken, Geb...",http://www.clubgoldwand.ch/,tel:056 282 30 50,NaN,Landstrasse 6\n5415 Nussbaumen AG,https://www.google.com/maps/place/Landstrasse%...,info@clubgoldwand.ch
4,NICOLE DIEM Horgen,Detailhandel mit Brillen und anderen Sehhilfen,http://www.nicolediem.ch/,tel:044 770 10 40,NaN,Dorfplatz 3\n8810 Horgen,https://www.google.com/maps/place/Dorfplatz%20...,payments@2x.png


In [67]:
df.tail()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
996,Martin Stoller Transporte,Güterbeförderung im Strassenverkehr,NaN,tel:031 741 24 15,NaN,Weidstrasse 2\n3184 Wünnewil,https://www.google.com/maps/place/Weidstrasse%...,website not accessible
997,Christoph Hasler Schreinerei,Schreinerarbeiten im Innenausbau,http://www.hasler-schreinerei.ch/,tel:071 917 11 55,fax:071 917 18 48,Käsereistrasse 1\n9555 Tobel,https://www.google.com/maps/place/K%C3%A4serei...,info@hasler-schreinerei.ch
998,MoRailSo AG,Grosshandel mit sonstigen Maschinen und Ausrüs...,https://www.ramorail.com/,tel:079 463 97 50,NaN,Rothmatte 7\n6022 Grosswangen,https://www.google.com/maps/place/Rothmatte%20...,bad link
999,Elektro Schuler AG,Elektroinstallation,NaN,tel:041 631 03 31,fax:041 631 03 32,Schulhausstrasse 4\n6052 Hergiswil NW,https://www.google.com/maps/place/Schulhausstr...,website not accessible
1000,Camping Lac des Brenets,Inserate,http://www.camping-brenets.ch/,tel:032 932 16 18,fax:032 932 16 39,2416 Les Brenets,https://www.google.com/maps/place/undefined+24...,info@camping-brenets.ch


In [68]:
df.head() == df1.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
0,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True
3,True,True,True,True,False,True,True,True
4,True,True,True,True,False,True,True,True


In [69]:
df.tail() == df4.tail()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
996,True,True,False,True,False,True,True,True
997,True,True,True,True,True,True,True,True
998,True,True,True,True,False,True,True,True
999,True,True,False,True,True,True,True,True
1000,True,True,True,True,True,True,True,True


### xxxxxxxxxxxx Push to Github xxxxxxxxxxxxxx

### Batch-5, batch_size = 1000, batch_start_index = 4001


#### !Cell is executed. Uncomment (Ctrl + A followed by Ctrl + /) and execute (Shift + Enter) again only if you want to run the batch again.

In [99]:
# # Extract info of interst
# start_time = time.time()
# batch5_initial = extract_info_of_interest(unique_company_profile_urls, batch_size = 1000, batch_start_index = 4001, sleep=5)
# end_time = time.time()
# execution_time = end_time - start_time
# print('execution time: ', execution_time)

count 1
count 2
count 3
count 4
count 5
count 6
count 7
count 8
count 9
count 10
count 11
count 12
count 13
count 14
count 15
count 16
count 17
count 18
count 19
count 20
count 21
count 22
count 23
count 24
count 25
count 26
count 27
count 28
count 29
count 30
count 31
count 32
count 33
count 34
count 35
count 36
count 37
count 38
count 39
count 40
count 41
count 42
count 43
count 44
count 45
count 46
count 47
count 48
count 49
count 50
count 51
count 52
count 53
count 54
count 55
count 56
count 57
count 58
count 59
count 60
count 61
count 62
count 63
count 64
count 65
count 66
count 67
count 68
count 69
count 70
count 71
count 72
count 73
count 74
count 75
count 76
count 77
count 78
count 79
count 80
count 81
count 82
count 83
count 84
count 85
count 86
count 87
count 88
count 89
count 90
count 91
count 92
count 93
count 94
count 95
count 96
count 97
count 98
count 99
count 100
count 101
count 102
count 103
count 104
count 105
count 106
count 107
count 108
count 109
count 110
count 11

count 832
count 833
count 834
count 835
count 836
count 837
count 838
count 839
count 840
count 841
count 842
count 843
count 844
count 845
count 846
count 847
count 848
count 849
count 850
count 851
count 852
count 853
count 854
count 855
count 856
count 857
count 858
count 859
count 860
count 861
count 862
count 863
count 864
count 865
count 866
count 867
count 868
count 869
count 870
count 871
count 872
count 873
count 874
count 875
count 876
count 877
count 878
count 879
count 880
count 881
count 882
count 883
count 884
count 885
count 886
count 887
count 888
count 889
count 890
count 891
count 892
count 893
count 894
count 895
count 896
count 897
count 898
count 899
count 900
count 901
count 902
count 903
count 904
count 905
count 906
count 907
count 908
count 909
count 910
count 911
count 912
count 913
count 914
count 915
count 916
count 917
count 918
count 919
count 920
count 921
count 922
count 923
count 924
count 925
count 926
count 927
count 928
count 929
count 930
count 931


### checkpoint-1

In [100]:
# # Test-Block !Uncomment and execute only if the batch is run again
# print(len(batch5_initial))
# print(len(batch5_initial[-1]))

1001
7


In [101]:
# # Test-Block !Uncomment and execute only if the batch is run again
# print(batch5_initial[:3])

[['Camping Lac des Brenets', 'Inserate', 'http://www.camping-brenets.ch/', 'tel:032 932 16 18', 'fax:032 932 16 39', '2416 Les Brenets', 'https://www.google.com/maps/place/undefined+2416+Les%20Brenets'], ['Elektro Capaul AG', 'Elektroinstallation', 'NA', 'tel:081 511 20 22', 'fax:081 511 20 21', 'Promenada 3\n7018 Flims Waldhaus', 'https://www.google.com/maps/place/Promenada%203+7018+Flims%20Waldhaus'], ['Parkhotel Bellevue & Spa', 'Hotels, Gasthöfe und Pensionen mit Restaurant', 'http://www.parkhotel-bellevue.ch/', 'tel:033 673 80 00', 'fax:033 673 80 01', 'Bellevuestrasse 15\n3715 Adelboden', 'https://www.google.com/maps/place/Bellevuestrasse%2015+3715+Adelboden']]


### End of Checkpoint-1

In [70]:
batch5df = pd.read_csv('batch5.csv')

In [71]:
batch5df_initial = batch5df.drop(['email'], axis = 1)

In [72]:
batch5df_initial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     1001 non-null   object
 1   industry         1001 non-null   object
 2   website          556 non-null    object
 3   telephone        893 non-null    object
 4   fax              570 non-null    object
 5   address          1001 non-null   object
 6   address_maplink  1001 non-null   object
dtypes: object(7)
memory usage: 54.9+ KB


In [73]:
batch5_initial = batch5df_initial.values.tolist()
print(batch1_initial[:3])

[['Ringler + Strahm Storenbau AG', 'Sonstige Bauinstallation', 'http://www.ringler-strahm.ch/', 'tel:033 345 22 55', 'fax:033 345 55 19', 'Uetendorfstrasse 20\n3634 Thierachern', 'https://www.google.com/maps/place/Uetendorfstrasse%2020+3634+Thierachern', 'info@ringler-strahm.ch'], ['Grob & Partner Architektur AG', 'Architekturbüros', 'http://www.grobarchitektur.ch/', 'tel:081 720 02 00', 'fax:081 720 02 05', 'Bahnhofstrasse 3\n7320 Sargans', 'https://www.google.com/maps/place/Bahnhofstrasse%20%203+7320+Sargans', '605a7baede844d278b89dc95ae0a9123@sentry-next.wixpress.com'], ['Lauclair AG', 'Schreinerarbeiten im Innenausbau', 'http://www.lauclair.ch/', 'tel:031 879 01 69', 'fax:031 879 20 69', 'Lyssstrasse 27\n3054 Schüpfen', 'https://www.google.com/maps/place/Lyssstrasse%2027+3054+Sch%C3%BCpfen', 'a95fca1f1eb9fe8g2c9ead0cd9931e2a@2x.jpg']]


#### Following Function Extracts emails from All Pages of a Site (Takes A Lot of Time: 7-15 sec per page)


In [74]:
# Extract emails
start_time = time.time()
pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
batch5 = extract_emails_from_allpages(batch5_initial, pattern = pattern , index=2)
end_time = time.time()
execution_time = end_time - start_time
print('execution time: ', execution_time)

C:\Users\Admin\miniconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


No of emails found:  324 ['info@camping-brenets.ch', 'info@bellevue-parkhotel.ch', 'idrosun@idrosun.ch', 'info@kaufmann-egg.ch', '605a7baede844d278b89dc95ae0a9123@sentry-next.wixpress.com', '605a7baede844d278b89dc95ae0a9123@sentry-next.wixpress.com', 'aareptan@gmail.com', 'kontakt@druckerei-gysin.ch', 'info@partyhardware.ch', 'bau@ac-fellner.ch', 'info@dama.ch', 'info@dudler.com', 'info@haepo.ch', 'dd0a55ccb8124b9c9d938e3acf41f8aa@sentry.wixpress.com', 'S6P_frontal_Logo2020_frei_web@2x.png', 'info@elektro-paganini.ch', '605a7baede844d278b89dc95ae0a9123@sentry-next.wixpress.com', 'info@venticlean.ch', 'zehnder@zehnderbau.ch', 'welcome@salzano.ch', 'bern@matter-ammann.ch', 'info@u-f.ch', 'info@fuchs-gerueste.ch', 'keineemail@keineemail.ch', 'contact@thermex.ch', 'SBFI_Logo@3x-1024x151.png', 'info@monsieurk.ch', 'info@domo-reisen.ch', 'info@muessig.ch', 'info@primuspublishing.ch', 'zeno.hug@bsi-software.com', 'info@chardonnens-boissons.ch', 'info@immoeiteberg.ch', 'r.chappuis@cas-gruppe.c

### Checkpoint-2

In [75]:
print(len(batch5)) # should be same as batch_size i.e. 1000
print(len(batch5[-1])) # should be 8
print(batch5[-5:]) #should have 8 items in each list with emails or remarks added

1001
8
[['Vinolatino GmbH', 'Grosshandel mit Wein und Spirituosen', nan, nan, nan, 'Hauptstrasse 33\n6280 Hochdorf', 'https://www.google.com/maps/place/Hauptstrasse%2033+6280+Hochdorf', 'website not accessible'], ['Cometal SA', 'Herstellung von Metallkonstruktionen', 'http://www.cometal.ch/', 'tel:091 859 10 22', 'fax:091 859 27 03', 'via Cantonale\n6595 Riazzino', 'https://www.google.com/maps/place/via%20Cantonale+6595+Riazzino', 'NA'], ['HCDM Beratungen GmbH', 'Unternehmensberatung', nan, 'tel:079 445 69 40', nan, 'Mühleweg 7\n7304 Maienfeld', 'https://www.google.com/maps/place/M%C3%BChleweg%207+7304+Maienfeld', 'website not accessible'], ['Eigenheer Elektro AG', 'Elektroinstallation', 'http://www.eigenheer-elektro.ch/', 'tel:052 317 13 79', nan, 'Burgstrasse 5\n8450 Andelfingen', 'https://www.google.com/maps/place/Burgstrasse%205+8450+Andelfingen', 'info@eigenheer-elektro.ch'], ['Papeterie Zumstein AG', 'Detailhandel mit Schreibwaren und Bürobedarf', nan, 'tel:044 211 77 70', nan, '

### End of Checkpoint-2

In [76]:
# # Convert to Dataframe batch1
df5 = pd.DataFrame(batch5, columns=columns)


### Checkpoint 3

In [77]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     1001 non-null   object
 1   industry         1001 non-null   object
 2   website          556 non-null    object
 3   telephone        893 non-null    object
 4   fax              570 non-null    object
 5   address          1001 non-null   object
 6   address_maplink  1001 non-null   object
 7   email_all        1001 non-null   object
dtypes: object(8)
memory usage: 62.7+ KB


In [78]:
df5.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
0,Camping Lac des Brenets,Inserate,http://www.camping-brenets.ch/,tel:032 932 16 18,fax:032 932 16 39,2416 Les Brenets,https://www.google.com/maps/place/undefined+24...,info@camping-brenets.ch
1,Elektro Capaul AG,Elektroinstallation,NaN,tel:081 511 20 22,fax:081 511 20 21,Promenada 3\n7018 Flims Waldhaus,https://www.google.com/maps/place/Promenada%20...,website not accessible
2,Parkhotel Bellevue & Spa,"Hotels, Gasthöfe und Pensionen mit Restaurant",http://www.parkhotel-bellevue.ch/,tel:033 673 80 00,fax:033 673 80 01,Bellevuestrasse 15\n3715 Adelboden,https://www.google.com/maps/place/Bellevuestra...,info@bellevue-parkhotel.ch
3,Supersaxo Gerold,Malerei und Gipserei ohne ausgeprägten Schwerp...,NaN,tel:g.supersaxo@valaiscom.ch,fax:027 957 45 16,Haus Rio\n3906 Saas-Fee,https://www.google.com/maps/place/Haus%20Rio+3...,website not accessible
4,Jenni Baumaschinen AG,"Grosshandel mit Bergwerks-, Bau- und Baustoffm...",http://www.rammax.ch/Joomla/,tel:041 920 36 62,fax:041 920 35 34,Haldenmattstrasse 2\n6210 Sursee,https://www.google.com/maps/place/Haldenmattst...,bad link


### End of Checkpoint-3

In [79]:
# save as csv
df5.to_csv('batch5_all.csv', index = False)

In [80]:
#### Merge the two dataframes
df = pd.concat([df, df5])

### Checkpoint-4

In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5005 entries, 0 to 1000
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     4976 non-null   object
 1   industry         4976 non-null   object
 2   website          2836 non-null   object
 3   telephone        4387 non-null   object
 4   fax              2779 non-null   object
 5   address          4976 non-null   object
 6   address_maplink  5005 non-null   object
 7   email_all        5005 non-null   object
dtypes: object(8)
memory usage: 351.9+ KB


In [82]:
df.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
0,Ringler + Strahm Storenbau AG,Sonstige Bauinstallation,http://www.ringler-strahm.ch/,tel:033 345 22 55,fax:033 345 55 19,Uetendorfstrasse 20\n3634 Thierachern,https://www.google.com/maps/place/Uetendorfstr...,info@ringler-strahm.ch
1,Grob & Partner Architektur AG,Architekturbüros,http://www.grobarchitektur.ch/,tel:081 720 02 00,fax:081 720 02 05,Bahnhofstrasse 3\n7320 Sargans,https://www.google.com/maps/place/Bahnhofstras...,605a7baede844d278b89dc95ae0a9123@sentry-next.w...
2,Lauclair AG,Schreinerarbeiten im Innenausbau,http://www.lauclair.ch/,tel:031 879 01 69,fax:031 879 20 69,Lyssstrasse 27\n3054 Schüpfen,https://www.google.com/maps/place/Lyssstrasse%...,a95fca1f1eb9fe8g2c9ead0cd9931e2a@2x.jpg
3,Club Goldwand,"Kauf und Verkauf von eigenen Grundstücken, Geb...",http://www.clubgoldwand.ch/,tel:056 282 30 50,NaN,Landstrasse 6\n5415 Nussbaumen AG,https://www.google.com/maps/place/Landstrasse%...,info@clubgoldwand.ch
4,NICOLE DIEM Horgen,Detailhandel mit Brillen und anderen Sehhilfen,http://www.nicolediem.ch/,tel:044 770 10 40,NaN,Dorfplatz 3\n8810 Horgen,https://www.google.com/maps/place/Dorfplatz%20...,payments@2x.png


In [83]:
df.tail()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
996,Vinolatino GmbH,Grosshandel mit Wein und Spirituosen,NaN,NaN,NaN,Hauptstrasse 33\n6280 Hochdorf,https://www.google.com/maps/place/Hauptstrasse...,website not accessible
997,Cometal SA,Herstellung von Metallkonstruktionen,http://www.cometal.ch/,tel:091 859 10 22,fax:091 859 27 03,via Cantonale\n6595 Riazzino,https://www.google.com/maps/place/via%20Canton...,NA
998,HCDM Beratungen GmbH,Unternehmensberatung,NaN,tel:079 445 69 40,NaN,Mühleweg 7\n7304 Maienfeld,https://www.google.com/maps/place/M%C3%BChlewe...,website not accessible
999,Eigenheer Elektro AG,Elektroinstallation,http://www.eigenheer-elektro.ch/,tel:052 317 13 79,NaN,Burgstrasse 5\n8450 Andelfingen,https://www.google.com/maps/place/Burgstrasse%...,info@eigenheer-elektro.ch
1000,Papeterie Zumstein AG,Detailhandel mit Schreibwaren und Bürobedarf,NaN,tel:044 211 77 70,NaN,Rennweg 19\n8001 Zürich,https://www.google.com/maps/place/Rennweg%2019...,website not accessible


In [84]:
df.head() == df1.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
0,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True
3,True,True,True,True,False,True,True,True
4,True,True,True,True,False,True,True,True


In [85]:
df.tail() == df5.tail()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
996,True,True,False,False,False,True,True,True
997,True,True,True,True,True,True,True,True
998,True,True,False,True,False,True,True,True
999,True,True,True,True,False,True,True,True
1000,True,True,False,True,False,True,True,True


### xxxxxxxxxxxx Push to Github xxxxxxxxxxxxxx

### Batch-6, batch_size = 1000, batch_start_index = 5001


#### !Cell is executed. Uncomment (Ctrl + A followed by Ctrl + /) and execute (Shift + Enter) again only if you want to run the batch again.

In [116]:
# # Extract info of interst
# start_time = time.time()
# batch6_initial = extract_info_of_interest(unique_company_profile_urls, batch_size = 1000, batch_start_index = 5001, sleep=5)
# end_time = time.time()
# execution_time = end_time - start_time
# print('execution time: ', execution_time)

count 1
count 2
count 3
count 4
count 5
count 6
count 7
count 8
count 9
count 10
count 11
count 12
count 13
count 14
count 15
count 16
count 17
count 18
count 19
count 20
count 21
count 22
count 23
count 24
count 25
count 26
count 27
count 28
count 29
count 30
count 31
count 32
count 33
count 34
count 35
count 36
count 37
count 38
count 39
count 40
count 41
count 42
count 43
count 44
count 45
count 46
count 47
count 48
count 49
count 50
count 51
count 52
count 53
count 54
count 55
count 56
count 57
count 58
count 59
count 60
count 61
count 62
count 63
count 64
count 65
count 66
count 67
count 68
count 69
count 70
count 71
count 72
count 73
count 74
count 75
count 76
count 77
count 78
count 79
count 80
count 81
count 82
count 83
count 84
count 85
count 86
count 87
count 88
count 89
count 90
count 91
count 92
count 93
count 94
count 95
count 96
count 97
count 98
count 99
count 100
count 101
count 102
count 103
count 104
count 105
count 106
count 107
count 108
count 109
count 110
count 11

count 832
count 833
count 834
count 835
count 836
count 837
count 838
count 839
count 840
count 841
count 842
count 843
count 844
count 845
count 846
count 847
count 848
count 849
count 850
count 851
count 852
count 853
count 854
count 855
count 856
count 857
count 858
count 859
count 860
count 861
count 862
count 863
count 864
count 865
count 866
count 867
count 868
count 869
count 870
count 871
count 872
count 873
count 874
count 875
count 876
count 877
count 878
count 879
count 880
count 881
count 882
count 883
count 884
count 885
count 886
count 887
count 888
count 889
count 890
count 891
count 892
count 893
count 894
count 895
count 896
count 897
count 898
count 899
count 900
count 901
count 902
count 903
count 904
count 905
count 906
count 907
count 908
count 909
count 910
count 911
count 912
count 913
count 914
count 915
count 916
count 917
count 918
count 919
count 920
count 921
count 922
count 923
count 924
count 925
count 926
count 927
count 928
count 929
count 930
count 931


### checkpoint-1

In [118]:
# # Test-Block !Uncomment and execute only if batch is run again
# print(len(batch6_initial))
# print(len(batch6_initial[-1]))

1001
7


In [119]:
# # Test-Block !Uncomment and execute only if batch is run again
# print(batch6_initial[:3])

[['Papeterie Zumstein AG', 'Detailhandel mit Schreibwaren und Bürobedarf', 'NA', 'tel:044 211 77 70', 'NA', 'Rennweg 19\n8001 Zürich', 'https://www.google.com/maps/place/Rennweg%2019+8001+Z%C3%BCrich'], ['Elektro Schuler AG', 'Elektroinstallation', 'NA', 'tel:041 612 06 33', 'NA', 'Forellenhof/Stanserstr.\n6362 Stansstad', 'https://www.google.com/maps/place/Forellenhof/Stanserstr.+6362+Stansstad'], ['Rüedi Büromaschinen', 'Detailhandel mit Telekommunikationsgeräten', 'http://www.rueedi-bueromaschinen.ch/', 'tel:034 415 19 23', 'fax:034 415 24 80', 'Gässli 2\n3473 Alchenstorf', 'https://www.google.com/maps/place/G%C3%A4ssli%202+3473+Alchenstorf']]


### End of Checkpoint-1

In [86]:
batch6df = pd.read_csv('batch6.csv')

In [87]:
batch6df_initial = batch6df.drop(['email'], axis = 1)

In [88]:
batch6df_initial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     999 non-null    object
 1   industry         1000 non-null   object
 2   website          575 non-null    object
 3   telephone        891 non-null    object
 4   fax              570 non-null    object
 5   address          999 non-null    object
 6   address_maplink  1001 non-null   object
dtypes: object(7)
memory usage: 54.9+ KB


In [89]:
batch6_initial = batch6df_initial.values.tolist()
print(batch6_initial[:3])

[['Papeterie Zumstein AG', 'Detailhandel mit Schreibwaren und Bürobedarf', nan, 'tel:044 211 77 70', nan, 'Rennweg 19\n8001 Zürich', 'https://www.google.com/maps/place/Rennweg%2019+8001+Z%C3%BCrich'], ['Elektro Schuler AG', 'Elektroinstallation', nan, 'tel:041 612 06 33', nan, 'Forellenhof/Stanserstr.\n6362 Stansstad', 'https://www.google.com/maps/place/Forellenhof/Stanserstr.+6362+Stansstad'], ['Rüedi Büromaschinen', 'Detailhandel mit Telekommunikationsgeräten', 'http://www.rueedi-bueromaschinen.ch/', 'tel:034 415 19 23', 'fax:034 415 24 80', 'Gässli 2\n3473 Alchenstorf', 'https://www.google.com/maps/place/G%C3%A4ssli%202+3473+Alchenstorf']]


#### Following Function Extracts emails from All Pages of a Site (Takes A Lot of Time: 7-15 sec per page)


In [90]:
# Extract emails
start_time = time.time()
pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
batch6 = extract_emails_from_allpages(batch6_initial, pattern = pattern , index=2)
end_time = time.time()
execution_time = end_time - start_time
print('execution time: ', execution_time)

C:\Users\Admin\miniconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


No of emails found:  375 ['info@max-mueller.ch', 'info@buerodesign.ch', 'info@kohlerundpartner.ch', '605a7baede844d278b89dc95ae0a9123@sentry-next.wixpress.com', 'info@immo-collection.ch', 'r.blanc@blanc-architekten.ch', 'info@bruggers.ch', 'info@uhrenstucky.ch', 'themen-rund-um-den-sandkasten_02_1280x1280@2x.jpg', 'info@straessle.com', 'info@gtl-sa.com', 'info@allodmathis.ch', 'mail@kuhn-gruppe.ch', 'info@vrp-machines.com', 'info@renus-ag.ch', 'info@schoetz.portas.ch', 'fg@digitalundprint.ch', 'info@willihaustechnik.ch', 'reto.schleiss@axa-winterthur.ch', 'ge_du@bluewin.ch', 'support@webador.de', 'info@stolzholz.ch', 'info@ib-automobile.ch', 'hotel@hotel-falken.ch', 'info@wenger-gartenbau.ch', 'raben.cham@bluewin.ch', 'weingut@hedinger.ch', 'support@delea.ch', 'Ivan@2x.jpg', 'DJI00602-2695633f-b926dc3f@214w.JPG', 'info@maler-grimm.ch', 'walter.mangold@garagebuser.ch', 'info@chromag.ch', 'info@schnyderag.ch', 'info@schwarzer-loewen.ch', 'donika.gjeloshi@blverlag.ch', 'info@gruber-polyba

### Checkpoint-2

In [91]:
print(len(batch6)) # should be same as batch_size i.e. 1000
print(len(batch6[-1])) # should be 8
print(batch6[-5:]) #should have 8 items in each list with emails or remarks added

1001
8
[['Max Auer', 'Vermietung und Verpachtung von eigenen oder geleasten Grundstücken', nan, 'tel:071 911 21 42', 'fax:071 911 85 27', 'Agathafeld 22\n9512 Rossrüti', 'https://www.google.com/maps/place/Agathafeld%2022+9512+Rossr%C3%BCti', 'website not accessible'], ['Rino Weder AG', 'Mechanische Werkstätten', 'http://www.rinoweder.ch/', 'tel:071 763 60 50', 'fax:071 763 60 51', 'Roggenweg 1\n9463 Oberriet SG', 'https://www.google.com/maps/place/Roggenweg%201+9463+Oberriet%20SG', 'verkauf@rinoweder.ch'], ['Garage Andermatt AG Baar', 'lnstandhaltung und Reparatur von Automobilen', 'http://www.garage-andermatt.ch/', 'tel:041 760 46 46', nan, 'Ruessenstrasse 22\n6340 Baar', 'https://www.google.com/maps/place/Ruessenstrasse%2022+6340+Baar', 'info@garage-andermatt.ch'], ['Pizzeria Mare Monte, Ekrem Sinani', 'Restaurants, Imbissstuben, Tea-Rooms und Gelaterias', nan, 'tel:071 845 60 06', 'fax:071 845 60 07', 'Blumenstrasse 8\n9403 Goldach', 'https://www.google.com/maps/place/Blumenstrasse%

### End of Checkpoint-2

In [92]:
# # Convert to Dataframe batch1
df6 = pd.DataFrame(batch6, columns=columns)


In [93]:
# # Convert to Dataframe batch1
df6 = pd.DataFrame(batch6, columns=columns)


### Checkpoint-3

In [94]:
df6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     999 non-null    object
 1   industry         1000 non-null   object
 2   website          575 non-null    object
 3   telephone        891 non-null    object
 4   fax              570 non-null    object
 5   address          999 non-null    object
 6   address_maplink  1001 non-null   object
 7   email_all        1001 non-null   object
dtypes: object(8)
memory usage: 62.7+ KB


In [95]:
# save as csv
df6.to_csv('batch6_all.csv', index = False)

### End of Checkpoint-3

In [96]:
#### Merge the two dataframes
df = pd.concat([df, df6])

### Checkpoint-4

In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6006 entries, 0 to 1000
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     5975 non-null   object
 1   industry         5976 non-null   object
 2   website          3411 non-null   object
 3   telephone        5278 non-null   object
 4   fax              3349 non-null   object
 5   address          5975 non-null   object
 6   address_maplink  6006 non-null   object
 7   email_all        6006 non-null   object
dtypes: object(8)
memory usage: 422.3+ KB


In [98]:
df.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
0,Ringler + Strahm Storenbau AG,Sonstige Bauinstallation,http://www.ringler-strahm.ch/,tel:033 345 22 55,fax:033 345 55 19,Uetendorfstrasse 20\n3634 Thierachern,https://www.google.com/maps/place/Uetendorfstr...,info@ringler-strahm.ch
1,Grob & Partner Architektur AG,Architekturbüros,http://www.grobarchitektur.ch/,tel:081 720 02 00,fax:081 720 02 05,Bahnhofstrasse 3\n7320 Sargans,https://www.google.com/maps/place/Bahnhofstras...,605a7baede844d278b89dc95ae0a9123@sentry-next.w...
2,Lauclair AG,Schreinerarbeiten im Innenausbau,http://www.lauclair.ch/,tel:031 879 01 69,fax:031 879 20 69,Lyssstrasse 27\n3054 Schüpfen,https://www.google.com/maps/place/Lyssstrasse%...,a95fca1f1eb9fe8g2c9ead0cd9931e2a@2x.jpg
3,Club Goldwand,"Kauf und Verkauf von eigenen Grundstücken, Geb...",http://www.clubgoldwand.ch/,tel:056 282 30 50,NaN,Landstrasse 6\n5415 Nussbaumen AG,https://www.google.com/maps/place/Landstrasse%...,info@clubgoldwand.ch
4,NICOLE DIEM Horgen,Detailhandel mit Brillen und anderen Sehhilfen,http://www.nicolediem.ch/,tel:044 770 10 40,NaN,Dorfplatz 3\n8810 Horgen,https://www.google.com/maps/place/Dorfplatz%20...,payments@2x.png


In [99]:
df.tail()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
996,Max Auer,Vermietung und Verpachtung von eigenen oder ge...,NaN,tel:071 911 21 42,fax:071 911 85 27,Agathafeld 22\n9512 Rossrüti,https://www.google.com/maps/place/Agathafeld%2...,website not accessible
997,Rino Weder AG,Mechanische Werkstätten,http://www.rinoweder.ch/,tel:071 763 60 50,fax:071 763 60 51,Roggenweg 1\n9463 Oberriet SG,https://www.google.com/maps/place/Roggenweg%20...,verkauf@rinoweder.ch
998,Garage Andermatt AG Baar,lnstandhaltung und Reparatur von Automobilen,http://www.garage-andermatt.ch/,tel:041 760 46 46,NaN,Ruessenstrasse 22\n6340 Baar,https://www.google.com/maps/place/Ruessenstras...,info@garage-andermatt.ch
999,"Pizzeria Mare Monte, Ekrem Sinani","Restaurants, Imbissstuben, Tea-Rooms und Gelat...",NaN,tel:071 845 60 06,fax:071 845 60 07,Blumenstrasse 8\n9403 Goldach,https://www.google.com/maps/place/Blumenstrass...,website not accessible
1000,TICINOCOLOR SA,Bewertungen,http://www.ticinocolor.ch/,tel:091 967 79 79,fax:091 967 79 81,via San Gottardo 146\n6942 Savosa,https://www.google.com/maps/place/via%20San%20...,NA


In [100]:
df.head()==df1.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
0,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True
3,True,True,True,True,False,True,True,True
4,True,True,True,True,False,True,True,True


In [101]:
df.tail()==df6.tail()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
996,True,True,False,True,True,True,True,True
997,True,True,True,True,True,True,True,True
998,True,True,True,True,False,True,True,True
999,True,True,False,True,True,True,True,True
1000,True,True,True,True,True,True,True,True


### End of Checkpoint-4

### xxxxxxxxxxxxxxxxxxxxx Push to Github xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

### Batch-7, batch_size = 1000, batch_start_index = 6001


#### !Cell is executed. Uncomment (Ctrl + A followed by Ctrl + /) and execute (Shift + Enter) again only if you want to run the batch again.

In [134]:
# # Extract info of interst
# start_time = time.time()
# batch7_initial = extract_info_of_interest(unique_company_profile_urls, batch_size = 1000, batch_start_index = 6001, sleep=5)
# end_time = time.time()
# execution_time = end_time - start_time
# print('execution time: ', execution_time)

count 1
count 2
count 3
count 4
count 5
count 6
count 7
count 8
count 9
count 10
count 11
count 12
count 13
count 14
count 15
count 16
count 17
count 18
count 19
count 20
count 21
count 22
count 23
count 24
count 25
count 26
count 27
count 28
count 29
count 30
count 31
count 32
count 33
count 34
count 35
count 36
count 37
count 38
count 39
count 40
count 41
count 42
count 43
count 44
count 45
count 46
count 47
count 48
count 49
count 50
count 51
count 52
count 53
count 54
count 55
count 56
count 57
count 58
count 59
count 60
count 61
count 62
count 63
count 64
count 65
count 66
count 67
count 68
count 69
count 70
count 71
count 72
count 73
count 74
count 75
count 76
count 77
count 78
count 79
count 80
count 81
count 82
count 83
count 84
count 85
count 86
count 87
count 88
count 89
count 90
count 91
count 92
count 93
count 94
count 95
count 96
count 97
count 98
count 99
count 100
count 101
count 102
count 103
count 104
count 105
count 106
count 107
count 108
count 109
count 110
count 11

count 832
count 833
count 834
count 835
count 836
count 837
count 838
count 839
count 840
count 841
count 842
count 843
count 844
count 845
count 846
count 847
count 848
count 849
count 850
count 851
count 852
count 853
count 854
count 855
count 856
count 857
count 858
count 859
count 860
count 861
count 862
count 863
count 864
count 865
count 866
count 867
count 868
count 869
count 870
count 871
count 872
count 873
count 874
count 875
count 876
count 877
count 878
count 879
count 880
count 881
count 882
count 883
count 884
count 885
count 886
count 887
count 888
count 889
count 890
count 891
count 892
count 893
count 894
count 895
count 896
count 897
count 898
count 899
count 900
count 901
count 902
count 903
count 904
count 905
count 906
count 907
count 908
count 909
count 910
count 911
count 912
count 913
count 914
count 915
count 916
count 917
count 918
count 919
count 920
count 921
count 922
count 923
count 924
count 925
count 926
count 927
count 928
count 929
count 930
count 931


### Checkpoint-1

In [135]:
# # Test-Block !Execute only if running the batch again
# print(len(batch7_initial))
# print(len(batch7_initial[-1]))

1001
7


In [136]:
# # Test-Block !Execute only if running the batch again
# print(batch7_initial[:3])

[['TICINOCOLOR SA', 'Bewertungen', 'http://www.ticinocolor.ch/', 'tel:091 967 79 79', 'fax:091 967 79 81', 'via San Gottardo 146\n6942 Savosa', 'https://www.google.com/maps/place/via%20San%20Gottardo%20146+6942+Savosa'], ['Amrein Reinigungen', 'Allgemeine Gebäudereinigung', 'NA', 'tel:062 771 88 70', 'fax:062 771 85 65', 'Alzbachstrasse 17\n5734 Reinach AG', 'https://www.google.com/maps/place/Alzbachstrasse%2017+5734+Reinach%20AG'], ['Duplirex Sensler Papeterie', 'Detailhandel mit Schreibwaren und Bürobedarf', 'NA', 'tel:026 493 33 55', 'NA', 'Hauptstrasse 17\n3186 Düdingen', 'https://www.google.com/maps/place/Hauptstrasse%2017+3186+D%C3%BCdingen']]


### End of Checkpoint-1

In [102]:
batch7df = pd.read_csv('batch7.csv')

In [103]:
batch7df_initial = batch7df.drop(['email'], axis = 1)

In [104]:
batch7df_initial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     1000 non-null   object
 1   industry         1001 non-null   object
 2   website          612 non-null    object
 3   telephone        878 non-null    object
 4   fax              562 non-null    object
 5   address          1000 non-null   object
 6   address_maplink  1001 non-null   object
dtypes: object(7)
memory usage: 54.9+ KB


In [105]:
batch7_initial = batch7df_initial.values.tolist()
print(batch1_initial[:3])

[['Ringler + Strahm Storenbau AG', 'Sonstige Bauinstallation', 'http://www.ringler-strahm.ch/', 'tel:033 345 22 55', 'fax:033 345 55 19', 'Uetendorfstrasse 20\n3634 Thierachern', 'https://www.google.com/maps/place/Uetendorfstrasse%2020+3634+Thierachern', 'info@ringler-strahm.ch'], ['Grob & Partner Architektur AG', 'Architekturbüros', 'http://www.grobarchitektur.ch/', 'tel:081 720 02 00', 'fax:081 720 02 05', 'Bahnhofstrasse 3\n7320 Sargans', 'https://www.google.com/maps/place/Bahnhofstrasse%20%203+7320+Sargans', '605a7baede844d278b89dc95ae0a9123@sentry-next.wixpress.com'], ['Lauclair AG', 'Schreinerarbeiten im Innenausbau', 'http://www.lauclair.ch/', 'tel:031 879 01 69', 'fax:031 879 20 69', 'Lyssstrasse 27\n3054 Schüpfen', 'https://www.google.com/maps/place/Lyssstrasse%2027+3054+Sch%C3%BCpfen', 'a95fca1f1eb9fe8g2c9ead0cd9931e2a@2x.jpg']]


#### Following Function Extracts emails from All Pages of a Site (Takes A Lot of Time: 7-15 sec per page)
 

In [106]:
# Extract emails
start_time = time.time()
pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
batch7 = extract_emails_from_allpages(batch7_initial, pattern = pattern , index=2)
end_time = time.time()
execution_time = end_time - start_time
print('execution time: ', execution_time)

C:\Users\Admin\miniconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


No of emails found:  383 ['markus@vinzenzpeter.ch', 'info@manufaktur6418.ch', 'info@oeno-tech.ch', 'info@hch-borer.ch', 'y.mueller@huber-messebau.ch', 'info@spagnologmbh.ch', 'info@fertiggaragen.ch', 'info@tribelhornag.ch', 'info@flueckigeroptik.ch', 'info@fwf.ch', 'info@winzelnberg.ch', 'info@drawag.ch', 'info@spectrum-gmbh.ch', 'info@t-kaeser.ch', 'info@mic-catering.ch', 'sandra.widemann@ppautotreff.ch', 'shop@blumenhaus-mueller.ch', 'info@jual.ch', 'info@mizy.ch', 'info@zysset-partner.ch', 'info@perosa.ch', 'hotel@cristalloarosa.ch', 'info@buvag.ch', 'info@swissdruck.ch', 'info@garage-roos.ch', 'info@ringo-tueren.ch', 'info@schreinerei-huber.ch', 'info@abag.ch', 'laiteriedumouret@bluewin.ch', 'info@goldenenkreuz.ch', 'info@ab-sirnach.ch', 'info@archi-nova.ch', 'lisa.schmidli@hegiag.ch', '605a7baede844d278b89dc95ae0a9123@sentry-next.wixpress.com', 'info@iplusag.ch', 'office@walz-backkunst.ch', 'team@getag.ch', 'info@heizmann-kunststoffe.ch', 'info@sichfreuen.ch', 'johndoe@domain.com'

### Checkpoint-2

In [107]:
print(len(batch7)) # should be same as batch_size i.e. 1000
print(len(batch7[-1])) # should be 8
print(batch7[:15]) #should have 8 items in each list with emails or remarks added

1001
8
[['TICINOCOLOR SA', 'Bewertungen', 'http://www.ticinocolor.ch/', 'tel:091 967 79 79', 'fax:091 967 79 81', 'via San Gottardo 146\n6942 Savosa', 'https://www.google.com/maps/place/via%20San%20Gottardo%20146+6942+Savosa', 'NA'], ['Amrein Reinigungen', 'Allgemeine Gebäudereinigung', nan, 'tel:062 771 88 70', 'fax:062 771 85 65', 'Alzbachstrasse 17\n5734 Reinach AG', 'https://www.google.com/maps/place/Alzbachstrasse%2017+5734+Reinach%20AG', 'website not accessible'], ['Duplirex Sensler Papeterie', 'Detailhandel mit Schreibwaren und Bürobedarf', nan, 'tel:026 493 33 55', nan, 'Hauptstrasse 17\n3186 Düdingen', 'https://www.google.com/maps/place/Hauptstrasse%2017+3186+D%C3%BCdingen', 'website not accessible'], ['Z-Audio Animatec AG', 'Grosshandel mit Geräten der Unterhaltungselektronik', 'http://www.z-audio.ch/', 'tel:044 370 20 40', 'fax:044 370 20 41', 'Schickmattweg 7\n8332 Russikon', 'https://www.google.com/maps/place/Schickmattweg%207+8332+Russikon', 'website not accessible'], ['V

### End of Checkpoint-2

In [108]:
# # Convert to Dataframe batch7
df7 = pd.DataFrame(batch7, columns=columns)


### Checkpoint-3

In [109]:
df7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     1000 non-null   object
 1   industry         1001 non-null   object
 2   website          612 non-null    object
 3   telephone        878 non-null    object
 4   fax              562 non-null    object
 5   address          1000 non-null   object
 6   address_maplink  1001 non-null   object
 7   email_all        1001 non-null   object
dtypes: object(8)
memory usage: 62.7+ KB


In [110]:
# save as csv
df7.to_csv('batch7_all.csv', index = False)

### End of Checkpoint-3

In [111]:
#### Merge the two dataframes
df = pd.concat([df, df7])

### Checkpoint-4

In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7007 entries, 0 to 1000
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     6975 non-null   object
 1   industry         6977 non-null   object
 2   website          4023 non-null   object
 3   telephone        6156 non-null   object
 4   fax              3911 non-null   object
 5   address          6975 non-null   object
 6   address_maplink  7007 non-null   object
 7   email_all        7007 non-null   object
dtypes: object(8)
memory usage: 492.7+ KB


In [113]:
df.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
0,Ringler + Strahm Storenbau AG,Sonstige Bauinstallation,http://www.ringler-strahm.ch/,tel:033 345 22 55,fax:033 345 55 19,Uetendorfstrasse 20\n3634 Thierachern,https://www.google.com/maps/place/Uetendorfstr...,info@ringler-strahm.ch
1,Grob & Partner Architektur AG,Architekturbüros,http://www.grobarchitektur.ch/,tel:081 720 02 00,fax:081 720 02 05,Bahnhofstrasse 3\n7320 Sargans,https://www.google.com/maps/place/Bahnhofstras...,605a7baede844d278b89dc95ae0a9123@sentry-next.w...
2,Lauclair AG,Schreinerarbeiten im Innenausbau,http://www.lauclair.ch/,tel:031 879 01 69,fax:031 879 20 69,Lyssstrasse 27\n3054 Schüpfen,https://www.google.com/maps/place/Lyssstrasse%...,a95fca1f1eb9fe8g2c9ead0cd9931e2a@2x.jpg
3,Club Goldwand,"Kauf und Verkauf von eigenen Grundstücken, Geb...",http://www.clubgoldwand.ch/,tel:056 282 30 50,NaN,Landstrasse 6\n5415 Nussbaumen AG,https://www.google.com/maps/place/Landstrasse%...,info@clubgoldwand.ch
4,NICOLE DIEM Horgen,Detailhandel mit Brillen und anderen Sehhilfen,http://www.nicolediem.ch/,tel:044 770 10 40,NaN,Dorfplatz 3\n8810 Horgen,https://www.google.com/maps/place/Dorfplatz%20...,payments@2x.png


In [114]:
df.tail()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
996,Garage du Lac F. Dougoud SA,Detailhandel mit Motorenkraftstoffen (Tankstel...,NaN,NaN,NaN,Rte de la Gruyère 115\n1644 Avry-devant-Pont,https://www.google.com/maps/place/Rte%20de%20l...,website not accessible
997,Bau GmbH André Bucher,"Anbringen von Stuckaturen, Gipserei und Verput...",NaN,NaN,NaN,Grossweid 28\n6026 Rain,https://www.google.com/maps/place/Grossweid%20...,website not accessible
998,"Stefan Thalmann AG, Baugeschäft,",Allgemeiner Hoch- und Tiefbau ohne ausgeprägte...,http://www.thalmann-bau.ch/,tel:041 660 43 68,fax:041 660 99 81,Dörfli 9\n6060 Ramersberg,https://www.google.com/maps/place/D%C3%B6rfli%...,mail@thalmann-bau.ch
999,Print-Fix Drucktechnik AG,Grosshandel mit Datenverarbeitungsgeräten und ...,NaN,tel:041 930 00 91,NaN,Krümmigasse 15\n6221 Rickenbach LU,https://www.google.com/maps/place/Kr%C3%BCmmig...,website not accessible
1000,Spenglerei Schmid GmbH,Bauspenglerei,http://www.spenglerei-schmid.ch/,NaN,NaN,Kesselstrasse 12\n8200 Schaffhausen,https://www.google.com/maps/place/Kesselstrass...,bad link


In [115]:
df.head()==df1.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
0,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True
3,True,True,True,True,False,True,True,True
4,True,True,True,True,False,True,True,True


In [116]:
df.tail()==df7.tail()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
996,True,True,False,False,False,True,True,True
997,True,True,False,False,False,True,True,True
998,True,True,True,True,True,True,True,True
999,True,True,False,True,False,True,True,True
1000,True,True,True,False,False,True,True,True


### xxxxxxxxxxxxxxx Push to Github xxxxxxxxxxxxxxxxxxxx

### Batch-8, batch_size = 1000, batch_start_index = 7001


#### !Cell is executed. Uncomment (Ctrl + A followed by Ctrl + /) and execute (Shift + Enter) again only if you want to run the batch again.

In [156]:
# # Extract info of interst
# start_time = time.time()
# batch8_initial = extract_info_of_interest(unique_company_profile_urls, batch_size = 1000, batch_start_index = 7001, sleep=5)
# end_time = time.time()
# execution_time = end_time - start_time
# print('execution time: ', execution_time)

count 1
count 2
count 3
count 4
count 5
count 6
count 7
count 8
count 9
count 10
count 11
count 12
count 13
count 14
count 15
count 16
count 17
count 18
count 19
count 20
count 21
count 22
count 23
count 24
count 25
count 26
count 27
count 28
count 29
count 30
count 31
count 32
count 33
count 34
count 35
count 36
count 37
count 38
count 39
count 40
count 41
count 42
count 43
count 44
count 45
count 46
count 47
count 48
count 49
count 50
count 51
count 52
count 53
count 54
count 55
count 56
count 57
count 58
count 59
count 60
count 61
count 62
count 63
count 64
count 65
count 66
count 67
count 68
count 69
count 70
count 71
count 72
count 73
count 74
count 75
count 76
count 77
count 78
count 79
count 80
count 81
count 82
count 83
count 84
count 85
count 86
count 87
count 88
count 89
count 90
count 91
count 92
count 93
count 94
count 95
count 96
count 97
count 98
count 99
count 100
count 101
count 102
count 103
count 104
count 105
count 106
count 107
count 108
count 109
count 110
count 11

count 832
count 833
count 834
count 835
count 836
count 837
count 838
count 839
count 840
count 841
count 842
count 843
count 844
count 845
count 846
count 847
count 848
count 849
count 850
count 851
count 852
count 853
count 854
count 855
count 856
count 857
count 858
count 859
count 860
count 861
count 862
count 863
count 864
count 865
count 866
count 867
count 868
count 869
count 870
count 871
count 872
count 873
count 874
count 875
count 876
count 877
count 878
count 879
count 880
count 881
count 882
count 883
count 884
count 885
count 886
count 887
count 888
count 889
count 890
count 891
count 892
count 893
count 894
count 895
count 896
count 897
count 898
count 899
count 900
count 901
count 902
count 903
count 904
count 905
count 906
count 907
count 908
count 909
count 910
count 911
count 912
count 913
count 914
count 915
count 916
count 917
count 918
count 919
count 920
count 921
count 922
count 923
count 924
count 925
count 926
count 927
count 928
count 929
count 930
count 931


### Checkpoint-1

In [ ]:
# # Test-Block !Uncomment and execute only if running batch again
# print(len(batch8_initial))
# print(len(batch8_initial[-1]))

In [ ]:
# # Test-Block !Uncomment and execute only if running batch again
# print(batch8_initial[:3])

### End of Checkpoint-1

In [117]:
batch8df = pd.read_csv('batch8.csv')

In [118]:
batch8df_initial = batch8df.drop(['email'], axis = 1)

In [119]:
batch8df_initial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     964 non-null    object
 1   industry         965 non-null    object
 2   website          570 non-null    object
 3   telephone        865 non-null    object
 4   fax              549 non-null    object
 5   address          964 non-null    object
 6   address_maplink  964 non-null    object
dtypes: object(7)
memory usage: 54.9+ KB


In [120]:
batch8_initial = batch8df_initial.values.tolist()
print(batch1_initial[:3])

[['Ringler + Strahm Storenbau AG', 'Sonstige Bauinstallation', 'http://www.ringler-strahm.ch/', 'tel:033 345 22 55', 'fax:033 345 55 19', 'Uetendorfstrasse 20\n3634 Thierachern', 'https://www.google.com/maps/place/Uetendorfstrasse%2020+3634+Thierachern', 'info@ringler-strahm.ch'], ['Grob & Partner Architektur AG', 'Architekturbüros', 'http://www.grobarchitektur.ch/', 'tel:081 720 02 00', 'fax:081 720 02 05', 'Bahnhofstrasse 3\n7320 Sargans', 'https://www.google.com/maps/place/Bahnhofstrasse%20%203+7320+Sargans', '605a7baede844d278b89dc95ae0a9123@sentry-next.wixpress.com'], ['Lauclair AG', 'Schreinerarbeiten im Innenausbau', 'http://www.lauclair.ch/', 'tel:031 879 01 69', 'fax:031 879 20 69', 'Lyssstrasse 27\n3054 Schüpfen', 'https://www.google.com/maps/place/Lyssstrasse%2027+3054+Sch%C3%BCpfen', 'a95fca1f1eb9fe8g2c9ead0cd9931e2a@2x.jpg']]


#### Following Function Extracts emails from All Pages of a Site (Takes A Lot of Time: 7-15 sec per page)
 

In [122]:
# Extract emails
start_time = time.time()
pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
batch8 = extract_emails_from_allpages(batch8_initial, pattern = pattern , index=2)
end_time = time.time()
execution_time = end_time - start_time

print('execution time: ', execution_time)

C:\Users\Admin\miniconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


No of emails found:  350 ['info@dorfplatz-zentrum-mm.ch', 'info@bidbern.ch', 'info@drim-gmbh.ch', 'info@automauchle.ch', 'support@dachcom.ch', 'ran2103@gmail.com', 'info@papeterie-steffisburg.ch', 'S6P_frontal_Logo2020_frei_web@2x.png', 'info@orpailleur.ch', 'info@flurygartenbau.ch', 'info@brennerjoehr.ch', 'info@us-car-center.ch', 'info@laube-sa.ch', 'raphael.michaelis@sigtech-ag.ch', 'dach-walser@bluewin.ch', 'info@wirtschaft-zum-schutz.ch', 'info@kgreiner.ch', '06blumen@blumen-flueckiger.ch', 'info@hofmannmechanik.ch', 'info@helfenstein-metallbau.ch', 'mail@kintec.ch', 'info@steiner-bauingenieure.ch', 'info@rcb.ch', 'info@btb.swiss', 'info@kellerverpackungstechnik.ch', 'info@promedag.ch', 'info@dachfenster.ch', 'he@hefemeubles.ch', 'info@lu-ag.ch', 'info@glasness.ch', 'info@effingermedien.ch', 'info@orpundgarage.ch', 'info@metzgergabriel.ch', 'info@kuenzi-haustechnik.ch', 'logo_@2X.png', 'reservation@artlevin.ch', 'info@ekb.ch', 'webmaster@juple.ch', 'pfaeffli@pfaeffli-elektro.ch', 

### Checkpoint-2

In [123]:
print(len(batch8)) # should be same as batch_size i.e. 1000
print(len(batch8[-1])) # should be 8
print(batch8[-5:]) #should have 8 items in each list with emails or remarks added

1001
8
[['Baumann Immobilien-+ Bau AG', 'Vermittlung von Grundstücken, Gebäuden und Wohnungen für Dritte', nan, 'tel:031 307 70 75', 'fax:031 307 70 73', 'Untere Zollgasse 28\n3072 Ostermundigen', 'https://www.google.com/maps/place/Untere%20Zollgasse%2028+3072+Ostermundigen', 'website not accessible'], ['Auto Baier AG', 'lnstandhaltung und Reparatur von Automobilen', 'http://www.auto-baier.ch/', 'tel:071 351 44 92', 'fax:071 351 44 82', 'Degersheimerstrasse 78\n9100 Herisau', 'https://www.google.com/maps/place/Degersheimerstrasse%2078+9100+Herisau', 'sekretariat@auto-baier.ch'], ['Ako-Clean AG', 'Allgemeine Gebäudereinigung', 'http://www.akogroup.ch/', 'tel:044 774 17 17', 'fax:044 774 17 16', 'Lenggenbachstrasse 3\n8951 Fahrweid', 'https://www.google.com/maps/place/Lenggenbachstrasse%203+8951+Fahrweid', 'info@akogroup.ch'], ['Daniel Kamber', 'Architekturbüros', 'http://www.kagi-architektur.ch/', 'tel:062 298 22 88', 'fax:062 298 34 83', 'Trottenackerstrasse 14\n4654 Lostorf', 'https:/

### End of Checkpoint-2

In [124]:
# # Convert to Dataframe batch8
df8 = pd.DataFrame(batch8, columns=columns)


### Checkpoint-3

In [125]:
df8.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     964 non-null    object
 1   industry         965 non-null    object
 2   website          570 non-null    object
 3   telephone        865 non-null    object
 4   fax              549 non-null    object
 5   address          964 non-null    object
 6   address_maplink  964 non-null    object
 7   email_all        1001 non-null   object
dtypes: object(8)
memory usage: 62.7+ KB


In [126]:
# save as csv
df8.to_csv('batch8_all.csv', index = False)

### End of Checkpoint-3

In [127]:
#### Merge the two dataframes
df = pd.concat([df, df8])

### Checkpoint-4

In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8008 entries, 0 to 1000
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     7939 non-null   object
 1   industry         7942 non-null   object
 2   website          4593 non-null   object
 3   telephone        7021 non-null   object
 4   fax              4460 non-null   object
 5   address          7939 non-null   object
 6   address_maplink  7971 non-null   object
 7   email_all        8008 non-null   object
dtypes: object(8)
memory usage: 563.1+ KB


In [129]:
df.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
0,Ringler + Strahm Storenbau AG,Sonstige Bauinstallation,http://www.ringler-strahm.ch/,tel:033 345 22 55,fax:033 345 55 19,Uetendorfstrasse 20\n3634 Thierachern,https://www.google.com/maps/place/Uetendorfstr...,info@ringler-strahm.ch
1,Grob & Partner Architektur AG,Architekturbüros,http://www.grobarchitektur.ch/,tel:081 720 02 00,fax:081 720 02 05,Bahnhofstrasse 3\n7320 Sargans,https://www.google.com/maps/place/Bahnhofstras...,605a7baede844d278b89dc95ae0a9123@sentry-next.w...
2,Lauclair AG,Schreinerarbeiten im Innenausbau,http://www.lauclair.ch/,tel:031 879 01 69,fax:031 879 20 69,Lyssstrasse 27\n3054 Schüpfen,https://www.google.com/maps/place/Lyssstrasse%...,a95fca1f1eb9fe8g2c9ead0cd9931e2a@2x.jpg
3,Club Goldwand,"Kauf und Verkauf von eigenen Grundstücken, Geb...",http://www.clubgoldwand.ch/,tel:056 282 30 50,NaN,Landstrasse 6\n5415 Nussbaumen AG,https://www.google.com/maps/place/Landstrasse%...,info@clubgoldwand.ch
4,NICOLE DIEM Horgen,Detailhandel mit Brillen und anderen Sehhilfen,http://www.nicolediem.ch/,tel:044 770 10 40,NaN,Dorfplatz 3\n8810 Horgen,https://www.google.com/maps/place/Dorfplatz%20...,payments@2x.png


In [130]:
df.head() == df1.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
0,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True
3,True,True,True,True,False,True,True,True
4,True,True,True,True,False,True,True,True


In [131]:
df.tail() == df8.tail()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
996,True,True,False,True,True,True,True,True
997,True,True,True,True,True,True,True,True
998,True,True,True,True,True,True,True,True
999,True,True,True,True,True,True,True,True
1000,True,True,False,False,False,True,True,True


### End of Checkpoint-4

### xxxxxxxxxxxxxxxxxxxxx Push to Github xxxxxxxxxxxxxxxxxxxxxxxxx

### Batch-9, batch_size = 1000, batch_start_index = 8001


#### !Cell is executed. Uncomment (Ctrl + A followed by Ctrl + /) and execute (Shift + Enter) again only if you want to run the batch again.

In [157]:
# # Extract info of interst
# start_time = time.time()
# batch9_initial = extract_info_of_interest(unique_company_profile_urls, batch_size = 1000, batch_start_index = 8001, sleep=5)
# end_time = time.time()
# execution_time = end_time - start_time
# print('execution time: ', execution_time)

count 1
count 2
count 3
count 4
count 5
count 6
count 7
count 8
count 9
count 10
count 11
count 12
count 13
count 14
count 15
count 16
count 17
count 18
count 19
count 20
count 21
count 22
count 23
count 24
count 25
count 26
count 27
count 28
count 29
count 30
count 31
count 32
count 33
count 34
count 35
count 36
count 37
count 38
count 39
count 40
count 41
count 42
count 43
count 44
count 45
count 46
count 47
count 48
count 49
count 50
count 51
count 52
count 53
count 54
count 55
count 56
count 57
count 58
count 59
count 60
count 61
count 62
count 63
count 64
count 65
count 66
count 67
count 68
count 69
count 70
count 71
count 72
count 73
count 74
count 75
count 76
count 77
count 78
count 79
count 80
count 81
count 82
count 83
count 84
count 85
count 86
count 87
count 88
count 89
count 90
count 91
count 92
count 93
count 94
count 95
count 96
count 97
count 98
count 99
count 100
count 101
count 102
count 103
count 104
count 105
count 106
count 107
count 108
count 109
count 110
count 11

count 832
count 833
count 834
count 835
count 836
count 837
count 838
count 839
count 840
count 841
count 842
count 843
count 844
count 845
count 846
count 847
count 848
count 849
count 850
count 851
count 852
count 853
count 854
count 855
count 856
count 857
count 858
count 859
count 860
count 861
count 862
count 863
count 864
count 865
count 866
count 867
count 868
count 869
count 870
count 871
count 872
count 873
count 874
count 875
count 876
count 877
count 878
count 879
count 880
count 881
count 882
count 883
count 884
count 885
count 886
count 887
count 888
count 889
count 890
count 891
count 892
count 893
count 894
count 895
count 896
count 897
count 898
count 899
count 900
count 901
count 902
count 903
count 904
count 905
count 906
count 907
count 908
count 909
count 910
count 911
count 912
count 913
count 914
count 915
count 916
count 917
count 918
count 919
count 920
count 921
count 922
count 923
count 924
count 925
count 926
count 927
count 928
count 929
count 930
count 931


### Checkpoint-1

In [ ]:
# # Test-Block !Uncomment and execute only if running the batch again
# print(len(batch9_initial))
# print(len(batch9_initial[-1]))

In [ ]:
# # Test-Block !Uncomment and execute only if running the batch again
# print(batch9_initial[:3])

### End of Checkpoint-1

In [132]:
batch9df = pd.read_csv('batch9.csv')

In [133]:
batch9df_initial = batch9df.drop(['email'], axis = 1)

In [134]:
batch9df_initial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     964 non-null    object
 1   industry         965 non-null    object
 2   website          570 non-null    object
 3   telephone        865 non-null    object
 4   fax              549 non-null    object
 5   address          964 non-null    object
 6   address_maplink  964 non-null    object
dtypes: object(7)
memory usage: 54.9+ KB


In [135]:
batch9_initial = batch9df_initial.values.tolist()
print(batch9_initial[:3])

[['Feger Wohnen AG', 'Bewertungen', 'http://www.moebelfeger.ch/', 'tel:071 446 25 46', 'fax:071 446 25 60', 'Rebhaldenstrasse 9\n9320 Arbon', 'https://www.google.com/maps/place/Rebhaldenstrasse%209+9320+Arbon'], ['Restaurant Höfli', 'Restaurants, Imbissstuben, Tea-Rooms und Gelaterias', 'http://www.dorfplatz-zentrum-mm.ch/', 'tel:033 346 00 70', nan, 'Dorfstrasse 12\n3662 Seftigen', 'https://www.google.com/maps/place/Dorfstrasse%2012+3662+Seftigen'], ['Werner Rieder', 'Sonstiges Körperpflegegewerbe', nan, 'tel:061 971 57 57', nan, 'Stücklerweg 1b\n4441 Thürnen', 'https://www.google.com/maps/place/St%C3%BCcklerweg%201b+4441+Th%C3%BCrnen']]


#### Following Function Extracts emails from All Pages of a Site (Takes A Lot of Time: 15 sec per page)


In [136]:
# Extract emails
start_time = time.time()
pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
batch9 = extract_emails_from_allpages(batch9_initial, pattern = pattern , index=2)
end_time = time.time()
execution_time = end_time - start_time
print('execution time: ', execution_time)

No of emails found:  237 ['info@dorfplatz-zentrum-mm.ch', 'info@bidbern.ch', 'info@drim-gmbh.ch', 'info@automauchle.ch', 'support@dachcom.ch', 'ran2103@gmail.com', 'info@papeterie-steffisburg.ch', 'S6P_frontal_Logo2020_frei_web@2x.png', 'info@orpailleur.ch', 'info@flurygartenbau.ch', 'info@brennerjoehr.ch', 'info@us-car-center.ch', 'info@laube-sa.ch', 'raphael.michaelis@sigtech-ag.ch', 'dach-walser@bluewin.ch', 'info@wirtschaft-zum-schutz.ch', 'info@kgreiner.ch', '06blumen@blumen-flueckiger.ch', 'info@hofmannmechanik.ch', 'info@helfenstein-metallbau.ch', 'mail@kintec.ch', 'info@steiner-bauingenieure.ch', 'info@rcb.ch', 'info@btb.swiss', 'info@kellerverpackungstechnik.ch', 'info@promedag.ch', 'info@dachfenster.ch', 'he@hefemeubles.ch', 'info@lu-ag.ch', 'info@glasness.ch', 'info@effingermedien.ch', 'info@orpundgarage.ch', 'info@metzgergabriel.ch', 'info@kuenzi-haustechnik.ch', 'logo_@2X.png', 'reservation@artlevin.ch', 'info@ekb.ch', 'webmaster@juple.ch', 'pfaeffli@pfaeffli-elektro.ch', 

### Checkpoint-2

In [ ]:
print(len(batch9)) # should be same as batch_size i.e. 1000
print(len(batch9[-1])) # should be 8
print(batch9[-5:]) #should have 8 items in each list with emails or remarks added

### End of Checkpoint-2

In [137]:
# # Convert to Dataframe batch9
df9 = pd.DataFrame(batch9, columns=columns)


### Checkpoint-3

In [138]:
df9.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     964 non-null    object
 1   industry         965 non-null    object
 2   website          570 non-null    object
 3   telephone        865 non-null    object
 4   fax              549 non-null    object
 5   address          964 non-null    object
 6   address_maplink  964 non-null    object
 7   email_all        1001 non-null   object
dtypes: object(8)
memory usage: 62.7+ KB


In [139]:
# save as csv
df9.to_csv('batch9_all.csv', index = False)

### End of Checkpoint-3

In [140]:
#### Merge the two dataframes
df = pd.concat([df, df9])

### Checkpoint-4

In [141]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9009 entries, 0 to 1000
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     8903 non-null   object
 1   industry         8907 non-null   object
 2   website          5163 non-null   object
 3   telephone        7886 non-null   object
 4   fax              5009 non-null   object
 5   address          8903 non-null   object
 6   address_maplink  8935 non-null   object
 7   email_all        9009 non-null   object
dtypes: object(8)
memory usage: 633.4+ KB


In [142]:
df.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
0,Ringler + Strahm Storenbau AG,Sonstige Bauinstallation,http://www.ringler-strahm.ch/,tel:033 345 22 55,fax:033 345 55 19,Uetendorfstrasse 20\n3634 Thierachern,https://www.google.com/maps/place/Uetendorfstr...,info@ringler-strahm.ch
1,Grob & Partner Architektur AG,Architekturbüros,http://www.grobarchitektur.ch/,tel:081 720 02 00,fax:081 720 02 05,Bahnhofstrasse 3\n7320 Sargans,https://www.google.com/maps/place/Bahnhofstras...,605a7baede844d278b89dc95ae0a9123@sentry-next.w...
2,Lauclair AG,Schreinerarbeiten im Innenausbau,http://www.lauclair.ch/,tel:031 879 01 69,fax:031 879 20 69,Lyssstrasse 27\n3054 Schüpfen,https://www.google.com/maps/place/Lyssstrasse%...,a95fca1f1eb9fe8g2c9ead0cd9931e2a@2x.jpg
3,Club Goldwand,"Kauf und Verkauf von eigenen Grundstücken, Geb...",http://www.clubgoldwand.ch/,tel:056 282 30 50,NaN,Landstrasse 6\n5415 Nussbaumen AG,https://www.google.com/maps/place/Landstrasse%...,info@clubgoldwand.ch
4,NICOLE DIEM Horgen,Detailhandel mit Brillen und anderen Sehhilfen,http://www.nicolediem.ch/,tel:044 770 10 40,NaN,Dorfplatz 3\n8810 Horgen,https://www.google.com/maps/place/Dorfplatz%20...,payments@2x.png


In [143]:
df.head() == df1.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
0,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True
3,True,True,True,True,False,True,True,True
4,True,True,True,True,False,True,True,True


In [144]:
df.tail() == df9.tail()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
996,True,True,False,True,True,True,True,True
997,True,True,True,True,True,True,True,True
998,True,True,True,True,True,True,True,True
999,True,True,True,True,True,True,True,True
1000,True,True,False,False,False,True,True,True


### End of Checkpoint-4

### xxxxxxxxxxxxxxxxxxxxxxxx Push to Github xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

### Batch-10, batch_size = 1000, batch_start_index = 9001


#### !Cell is executed. Uncomment (Ctrl + A followed by Ctrl + /) and execute (Shift + Enter) again only if you want to run the batch again.

In [22]:
# # Extract info of interst
# start_time = time.time()
# batch10_initial = extract_info_of_interest(unique_company_profile_urls, batch_size = 1000, batch_start_index = 9001, sleep=5)
# end_time = time.time()
# execution_time = end_time - start_time
# print('execution time: ', execution_time)

count 1
count 2
count 3
count 4
count 5
count 6
count 7
count 8
count 9
count 10
count 11
count 12
count 13
count 14
count 15
count 16
count 17
count 18
count 19
count 20
count 21
count 22
count 23
count 24
count 25
count 26
count 27
count 28
count 29
count 30
count 31
count 32
count 33
count 34
count 35
count 36
count 37
count 38
count 39
count 40
count 41
count 42
count 43
count 44
count 45
count 46
count 47
count 48
count 49
count 50
count 51
count 52
count 53
count 54
count 55
count 56
count 57
count 58
count 59
count 60
count 61
count 62
count 63
count 64
count 65
count 66
count 67
count 68
count 69
count 70
count 71
count 72
count 73
count 74
count 75
count 76
count 77
count 78
count 79
count 80
count 81
count 82
count 83
count 84
count 85
count 86
count 87
count 88
count 89
count 90
count 91
count 92
count 93
count 94
count 95
count 96
count 97
count 98
count 99
count 100
count 101
count 102
count 103
count 104
count 105
count 106
count 107
count 108
count 109
count 110
count 11

count 832
count 833
count 834
count 835
count 836
count 837
count 838
count 839
count 840
count 841
count 842
count 843
count 844
count 845
count 846
count 847
count 848
count 849
count 850
count 851
count 852
count 853
count 854
count 855
count 856
count 857
count 858
count 859
count 860
count 861
count 862
count 863
count 864
count 865
count 866
count 867
count 868
count 869
count 870
count 871
count 872
count 873
count 874
count 875
count 876
count 877
count 878
count 879
count 880
count 881
count 882
count 883
count 884
count 885
count 886
count 887
count 888
count 889
count 890
count 891
count 892
count 893
count 894
count 895
count 896
count 897
count 898
count 899
count 900
count 901
count 902
count 903
count 904
count 905
count 906
count 907
count 908
count 909
count 910
count 911
count 912
count 913
count 914
count 915
count 916
count 917
count 918
count 919
count 920
count 921
count 922
count 923
count 924
count 925
count 926
count 927
count 928
count 929
count 930
count 931


### Checkpoint-1

In [ ]:
# # Test-Block !Uncomment and execute only if running the batch again
# print(len(batch10_initial))
# print(len(batch10_initial[-1]))

In [ ]:
# # Test-Block !Uncomment and execute only if running the batch again
# print(batch10_initial[:3])

### End of Checkpoint-1

In [145]:
batch10df = pd.read_csv('batch10.csv')

In [146]:
batch10df_initial = batch10df.drop(['email'], axis = 1)

In [147]:
batch10df_initial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     964 non-null    object
 1   industry         965 non-null    object
 2   website          570 non-null    object
 3   telephone        865 non-null    object
 4   fax              549 non-null    object
 5   address          964 non-null    object
 6   address_maplink  964 non-null    object
dtypes: object(7)
memory usage: 54.9+ KB


In [148]:
batch10_initial = batch10df_initial.values.tolist()
print(batch10_initial[:3])

[['Feger Wohnen AG', 'Bewertungen', 'http://www.moebelfeger.ch/', 'tel:071 446 25 46', 'fax:071 446 25 60', 'Rebhaldenstrasse 9\n9320 Arbon', 'https://www.google.com/maps/place/Rebhaldenstrasse%209+9320+Arbon'], ['Restaurant Höfli', 'Restaurants, Imbissstuben, Tea-Rooms und Gelaterias', 'http://www.dorfplatz-zentrum-mm.ch/', 'tel:033 346 00 70', nan, 'Dorfstrasse 12\n3662 Seftigen', 'https://www.google.com/maps/place/Dorfstrasse%2012+3662+Seftigen'], ['Werner Rieder', 'Sonstiges Körperpflegegewerbe', nan, 'tel:061 971 57 57', nan, 'Stücklerweg 1b\n4441 Thürnen', 'https://www.google.com/maps/place/St%C3%BCcklerweg%201b+4441+Th%C3%BCrnen']]


#### Following Function Extracts emails from All Pages of a Site (Takes A Lot of Time: 7-15 sec per page)


In [150]:
# Extract emails
start_time = time.time()
pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
batch10 = extract_emails_from_allpages(batch10_initial, pattern = pattern , index=2)
end_time = time.time()
execution_time = end_time - start_time
print('execution time: ', execution_time)

C:\Users\Admin\miniconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


No of emails found:  352 ['info@dorfplatz-zentrum-mm.ch', 'info@bidbern.ch', 'info@drim-gmbh.ch', 'info@automauchle.ch', 'support@dachcom.ch', 'ran2103@gmail.com', 'info@papeterie-steffisburg.ch', 'S6P_frontal_Logo2020_frei_web@2x.png', 'info@orpailleur.ch', 'info@flurygartenbau.ch', 'info@brennerjoehr.ch', 'info@us-car-center.ch', 'info@laube-sa.ch', 'raphael.michaelis@sigtech-ag.ch', 'dach-walser@bluewin.ch', 'info@wirtschaft-zum-schutz.ch', 'info@kgreiner.ch', '06blumen@blumen-flueckiger.ch', 'info@hofmannmechanik.ch', 'info@helfenstein-metallbau.ch', 'mail@kintec.ch', 'info@steiner-bauingenieure.ch', 'info@rcb.ch', 'info@btb.swiss', 'info@kellerverpackungstechnik.ch', 'info@promedag.ch', 'info@dachfenster.ch', 'he@hefemeubles.ch', 'info@lu-ag.ch', 'info@glasness.ch', 'info@effingermedien.ch', 'info@orpundgarage.ch', 'info@metzgergabriel.ch', 'info@kuenzi-haustechnik.ch', 'logo_@2X.png', 'reservation@artlevin.ch', 'info@ekb.ch', 'webmaster@juple.ch', 'pfaeffli@pfaeffli-elektro.ch', 

### Checkpoint-2

In [151]:
print(len(batch10)) # should be same as batch_size i.e. 1000
print(len(batch10[-1])) # should be 8
print(batch10[-5:]) #should have 8 items in each list with emails or remarks added

1001
8
[['Baumann Immobilien-+ Bau AG', 'Vermittlung von Grundstücken, Gebäuden und Wohnungen für Dritte', nan, 'tel:031 307 70 75', 'fax:031 307 70 73', 'Untere Zollgasse 28\n3072 Ostermundigen', 'https://www.google.com/maps/place/Untere%20Zollgasse%2028+3072+Ostermundigen', 'website not accessible'], ['Auto Baier AG', 'lnstandhaltung und Reparatur von Automobilen', 'http://www.auto-baier.ch/', 'tel:071 351 44 92', 'fax:071 351 44 82', 'Degersheimerstrasse 78\n9100 Herisau', 'https://www.google.com/maps/place/Degersheimerstrasse%2078+9100+Herisau', 'sekretariat@auto-baier.ch'], ['Ako-Clean AG', 'Allgemeine Gebäudereinigung', 'http://www.akogroup.ch/', 'tel:044 774 17 17', 'fax:044 774 17 16', 'Lenggenbachstrasse 3\n8951 Fahrweid', 'https://www.google.com/maps/place/Lenggenbachstrasse%203+8951+Fahrweid', 'info@akogroup.ch'], ['Daniel Kamber', 'Architekturbüros', 'http://www.kagi-architektur.ch/', 'tel:062 298 22 88', 'fax:062 298 34 83', 'Trottenackerstrasse 14\n4654 Lostorf', 'https:/

### End of Checkpoint-2

In [152]:
# # Convert to Dataframe batch10
df10 = pd.DataFrame(batch10, columns=columns)


### Checkpoint-3

In [153]:
df10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     964 non-null    object
 1   industry         965 non-null    object
 2   website          570 non-null    object
 3   telephone        865 non-null    object
 4   fax              549 non-null    object
 5   address          964 non-null    object
 6   address_maplink  964 non-null    object
 7   email_all        1001 non-null   object
dtypes: object(8)
memory usage: 62.7+ KB


In [154]:
# save as csv
df10.to_csv('batch10_all.csv', index = False)

### End of Checkpoint-3

In [155]:
#### Merge the two dataframes
df = pd.concat([df, df10])

### Checkpoint-4

In [156]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10010 entries, 0 to 1000
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     9867 non-null   object
 1   industry         9872 non-null   object
 2   website          5733 non-null   object
 3   telephone        8751 non-null   object
 4   fax              5558 non-null   object
 5   address          9867 non-null   object
 6   address_maplink  9899 non-null   object
 7   email_all        10010 non-null  object
dtypes: object(8)
memory usage: 703.8+ KB


In [157]:
df.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
0,Ringler + Strahm Storenbau AG,Sonstige Bauinstallation,http://www.ringler-strahm.ch/,tel:033 345 22 55,fax:033 345 55 19,Uetendorfstrasse 20\n3634 Thierachern,https://www.google.com/maps/place/Uetendorfstr...,info@ringler-strahm.ch
1,Grob & Partner Architektur AG,Architekturbüros,http://www.grobarchitektur.ch/,tel:081 720 02 00,fax:081 720 02 05,Bahnhofstrasse 3\n7320 Sargans,https://www.google.com/maps/place/Bahnhofstras...,605a7baede844d278b89dc95ae0a9123@sentry-next.w...
2,Lauclair AG,Schreinerarbeiten im Innenausbau,http://www.lauclair.ch/,tel:031 879 01 69,fax:031 879 20 69,Lyssstrasse 27\n3054 Schüpfen,https://www.google.com/maps/place/Lyssstrasse%...,a95fca1f1eb9fe8g2c9ead0cd9931e2a@2x.jpg
3,Club Goldwand,"Kauf und Verkauf von eigenen Grundstücken, Geb...",http://www.clubgoldwand.ch/,tel:056 282 30 50,NaN,Landstrasse 6\n5415 Nussbaumen AG,https://www.google.com/maps/place/Landstrasse%...,info@clubgoldwand.ch
4,NICOLE DIEM Horgen,Detailhandel mit Brillen und anderen Sehhilfen,http://www.nicolediem.ch/,tel:044 770 10 40,NaN,Dorfplatz 3\n8810 Horgen,https://www.google.com/maps/place/Dorfplatz%20...,payments@2x.png


In [158]:
df.tail()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
996,Baumann Immobilien-+ Bau AG,"Vermittlung von Grundstücken, Gebäuden und Woh...",NaN,tel:031 307 70 75,fax:031 307 70 73,Untere Zollgasse 28\n3072 Ostermundigen,https://www.google.com/maps/place/Untere%20Zol...,website not accessible
997,Auto Baier AG,lnstandhaltung und Reparatur von Automobilen,http://www.auto-baier.ch/,tel:071 351 44 92,fax:071 351 44 82,Degersheimerstrasse 78\n9100 Herisau,https://www.google.com/maps/place/Degersheimer...,sekretariat@auto-baier.ch
998,Ako-Clean AG,Allgemeine Gebäudereinigung,http://www.akogroup.ch/,tel:044 774 17 17,fax:044 774 17 16,Lenggenbachstrasse 3\n8951 Fahrweid,https://www.google.com/maps/place/Lenggenbachs...,info@akogroup.ch
999,Daniel Kamber,Architekturbüros,http://www.kagi-architektur.ch/,tel:062 298 22 88,fax:062 298 34 83,Trottenackerstrasse 14\n4654 Lostorf,https://www.google.com/maps/place/Trottenacker...,kagi54@bluewin.ch
1000,c+j immo AG,"Kauf und Verkauf von eigenen Grundstücken, Geb...",NaN,NaN,NaN,Industriestrasse 21\n6055 Alpnach Dorf,https://www.google.com/maps/place/Industriestr...,website not accessible


In [159]:
df.head() == df1.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
0,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True
3,True,True,True,True,False,True,True,True
4,True,True,True,True,False,True,True,True


In [160]:
df.tail() == df10.tail()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
996,True,True,False,True,True,True,True,True
997,True,True,True,True,True,True,True,True
998,True,True,True,True,True,True,True,True
999,True,True,True,True,True,True,True,True
1000,True,True,False,False,False,True,True,True


### End of Checkpoint-4

### xxxxxxxxxxxxxxxxxxx Push to Github xxxxxxxxxxxxxxxxxxxxxxxx

### Batch-11, batch_size = 1000, batch_start_index = 10001


#### !Cell is executed. Uncomment (Ctrl + A followed by Ctrl + /) and execute (Shift + Enter) again only if you want to run the batch again.

In [34]:
# # Extract info of interst
# start_time = time.time()
# batch11_initial = extract_info_of_interest(unique_company_profile_urls, batch_size = 1000, batch_start_index = 10001, sleep=5)
# end_time = time.time()
# execution_time = end_time - start_time
# print('execution time: ', execution_time)

count 1
count 2
count 3
count 4
count 5
count 6
count 7
count 8
count 9
count 10
count 11
count 12
count 13
count 14
count 15
count 16
count 17
count 18
count 19
count 20
count 21
count 22
count 23
count 24
count 25
count 26
count 27
count 28
count 29
count 30
count 31
count 32
count 33
count 34
count 35
count 36
count 37
count 38
count 39
count 40
count 41
count 42
count 43
count 44
count 45
count 46
count 47
count 48
count 49
count 50
count 51
count 52
count 53
count 54
count 55
count 56
count 57
count 58
count 59
count 60
count 61
count 62
count 63
count 64
count 65
count 66
count 67
count 68
count 69
count 70
count 71
count 72
count 73
count 74
count 75
count 76
count 77
count 78
count 79
count 80
count 81
count 82
count 83
count 84
count 85
count 86
count 87
count 88
count 89
count 90
count 91
count 92
count 93
count 94
count 95
count 96
count 97
count 98
count 99
count 100
count 101
count 102
count 103
count 104
count 105
count 106
count 107
count 108
count 109
count 110
count 11

count 832
count 833
count 834
count 835
count 836
count 837
count 838
count 839
count 840
count 841
count 842
count 843
count 844
count 845
count 846
count 847
count 848
count 849
count 850
count 851
count 852
count 853
count 854
count 855
count 856
count 857
count 858
count 859
count 860
count 861
count 862
count 863
count 864
count 865
count 866
count 867
count 868
count 869
count 870
count 871
count 872
count 873
count 874
count 875
count 876
count 877
count 878
count 879
count 880
count 881
count 882
count 883
count 884
count 885
count 886
count 887
count 888
count 889
count 890
count 891
count 892
count 893
count 894
count 895
count 896
count 897
count 898
count 899
count 900
count 901
count 902
count 903
count 904
count 905
count 906
count 907
count 908
count 909
count 910
count 911
count 912
count 913
count 914
count 915
count 916
count 917
count 918
count 919
count 920
count 921
count 922
count 923
count 924
count 925
count 926
count 927
count 928
count 929
count 930
count 931


### Checkpoint-1

In [35]:
# # Test-Block !Uncomment and execute only if running the batch again
# print(len(batch11_initial))
# print(len(batch11_initial[-1]))

1001
7


In [36]:
# # # Test-Block !Uncomment and execute only if running the batch again
# print(batch11_initial[:3])

[['c+j immo AG', 'Kauf und Verkauf von eigenen Grundstücken, Gebäuden und Wohnungen', 'NA', 'NA', 'NA', 'Industriestrasse 21\n6055 Alpnach Dorf', 'https://www.google.com/maps/place/Industriestrasse%2021+6055+Alpnach%20Dorf'], ['Dr.iur. Marco Ettisberger', 'Advokatur-, Notariatsbüros', 'http://www.eplaw.ch/', 'tel:081 254 38 00', 'fax:081 254 38 09', 'Hinterm Bach 40\n7002 Chur', 'https://www.google.com/maps/place/Hinterm%20Bach%2040+7002+Chur'], ['Techno AG', 'Grosshandel mit Automobilteilen und -zubehör', 'NA', 'tel:061 717 90 00', 'fax:061 711 38 58', 'Butthollenring 31\n4147 Aesch BL', 'https://www.google.com/maps/place/Butthollenring%2031+4147+Aesch%20BL']]


### End of Checkpoint-1

In [161]:
batch11df = pd.read_csv('batch11.csv')

In [162]:
batch11df_initial = batch11df.drop(['email'], axis = 1)

In [163]:
batch11df_initial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     1000 non-null   object
 1   industry         1001 non-null   object
 2   website          546 non-null    object
 3   telephone        895 non-null    object
 4   fax              549 non-null    object
 5   address          1000 non-null   object
 6   address_maplink  1000 non-null   object
dtypes: object(7)
memory usage: 54.9+ KB


In [164]:
batch11_initial = batch11df_initial.values.tolist()
print(batch11_initial[:3])

[['c+j immo AG', 'Kauf und Verkauf von eigenen Grundstücken, Gebäuden und Wohnungen', nan, nan, nan, 'Industriestrasse 21\n6055 Alpnach Dorf', 'https://www.google.com/maps/place/Industriestrasse%2021+6055+Alpnach%20Dorf'], ['Dr.iur. Marco Ettisberger', 'Advokatur-, Notariatsbüros', 'http://www.eplaw.ch/', 'tel:081 254 38 00', 'fax:081 254 38 09', 'Hinterm Bach 40\n7002 Chur', 'https://www.google.com/maps/place/Hinterm%20Bach%2040+7002+Chur'], ['Techno AG', 'Grosshandel mit Automobilteilen und -zubehör', nan, 'tel:061 717 90 00', 'fax:061 711 38 58', 'Butthollenring 31\n4147 Aesch BL', 'https://www.google.com/maps/place/Butthollenring%2031+4147+Aesch%20BL']]


#### Following Function Extracts emails from All Pages of a Site (Takes A Lot of Time: 7-15 sec per page)


In [165]:
# Extract emails
start_time = time.time()
pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
batch11 = extract_emails_from_allpages(batch11_initial, pattern = pattern , index=2)
end_time = time.time()
execution_time = end_time - start_time
print('execution time: ', execution_time)

No of emails found:  328 ['hello@village-project.org', 'info@kuenzleschreinerei.ch', 'info@ablschweiz.ch', '605a7baede844d278b89dc95ae0a9123@sentry-next.wixpress.com', '605a7baede844d278b89dc95ae0a9123@sentry-next.wixpress.com', 'info@suissepromotion.ch', 'info@fona.ch', 'mail@schneider-metall.ch', 'info@hugentoblerheizungen.ch', 'hallo@huegli.swiss', 'info@stilgrenze.de', 'info@senn-parkett.ch', 'info@artho-ag.ch', 'info@schnyder-parkett.ch', 'info@vogel-galabau.ch', 'info@schuh-interlaken.ch', 'info@rolf-weidmann.ch', 'info@edwinhildbrand.ch', 'info@car-point-lyss.ch', 'info@curtolo.ch', 'bild1-58e86b0290f5583g7dc3ea206bdb93c1@2x.jpg', 'info@haganatur.ch', 'info@aarcalit-handels.ch', 'info@atelier19.ch', 'info@autovonah.ch', 'werder.gmbh@bluewin.ch', 'mgisin@ws-skyworker.ch', 'pascal@fiduciairegilomen.ch', '605a7baede844d278b89dc95ae0a9123@sentry-next.wixpress.com', 'info@telcomnet.ch', 'info@neumet.ch', 'baden@elinag.ch', 'info@garage-meili.ch', 'info@spiessag.com', 'info@wittwer-tr

### Checkpoint-2

In [166]:
print(len(batch11)) # should be same as batch_size i.e. 1000
print(len(batch11[-1])) # should be 8
print(batch11[-5:]) #should have 8 items in each list with emails or remarks added

1001
8
[['Büro Weber AG', 'Detailhandel mit Möbeln', 'http://www.buero-weber.ch/', 'tel:056 622 20 50', 'fax:056 622 86 64', 'Gewerbering 23\n5610 Wohlen AG', 'https://www.google.com/maps/place/Gewerbering%2023+5610+Wohlen%20AG', 'NA'], ['W. Kamm AG', 'lnstallation von Heizungs-, Lüftungs- und Klimaanlagen', nan, 'tel:071 971 10 55', nan, 'Bahnhofstrasse 3\n8360 Eschlikon TG', 'https://www.google.com/maps/place/Bahnhofstrasse%203+8360+Eschlikon%20TG', 'website not accessible'], ['Mebrag GmbH', 'Herstellung von Metallkonstruktionen', 'http://www.mebrag.ch/', 'tel:032 387 25 70', nan, 'Mühleweg 6\n3253 Schnottwil', 'https://www.google.com/maps/place/M%C3%BChleweg%206+3253+Schnottwil', 'NA'], ['Garage Elmiger AG', 'Detailhandel mit Motorenkraftstoffen (Tankstellen)', nan, nan, nan, 'Luzernerstrasse 32\n6285 Hitzkirch', 'https://www.google.com/maps/place/Luzernerstrasse%2032+6285+Hitzkirch', 'website not accessible'], ['Sajade AG', 'Sonstiger Detailhandel mit Metallwaren, Anstrichmitteln, 

### End of Checkpoint-2

In [167]:
# # Convert to Dataframe batch11
df11 = pd.DataFrame(batch11, columns=columns)


### Checkpoint-3

In [168]:
df11.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     1000 non-null   object
 1   industry         1001 non-null   object
 2   website          546 non-null    object
 3   telephone        895 non-null    object
 4   fax              549 non-null    object
 5   address          1000 non-null   object
 6   address_maplink  1000 non-null   object
 7   email_all        1001 non-null   object
dtypes: object(8)
memory usage: 62.7+ KB


In [169]:
# save as csv
df11.to_csv('batch11_all.csv', index = False)

### End of Checkpoint-3

In [170]:
#### Merge the two dataframes
df = pd.concat([df, df11])

### Checkpoint-4

In [171]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11011 entries, 0 to 1000
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     10867 non-null  object
 1   industry         10873 non-null  object
 2   website          6279 non-null   object
 3   telephone        9646 non-null   object
 4   fax              6107 non-null   object
 5   address          10867 non-null  object
 6   address_maplink  10899 non-null  object
 7   email_all        11011 non-null  object
dtypes: object(8)
memory usage: 774.2+ KB


In [172]:
df.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
0,Ringler + Strahm Storenbau AG,Sonstige Bauinstallation,http://www.ringler-strahm.ch/,tel:033 345 22 55,fax:033 345 55 19,Uetendorfstrasse 20\n3634 Thierachern,https://www.google.com/maps/place/Uetendorfstr...,info@ringler-strahm.ch
1,Grob & Partner Architektur AG,Architekturbüros,http://www.grobarchitektur.ch/,tel:081 720 02 00,fax:081 720 02 05,Bahnhofstrasse 3\n7320 Sargans,https://www.google.com/maps/place/Bahnhofstras...,605a7baede844d278b89dc95ae0a9123@sentry-next.w...
2,Lauclair AG,Schreinerarbeiten im Innenausbau,http://www.lauclair.ch/,tel:031 879 01 69,fax:031 879 20 69,Lyssstrasse 27\n3054 Schüpfen,https://www.google.com/maps/place/Lyssstrasse%...,a95fca1f1eb9fe8g2c9ead0cd9931e2a@2x.jpg
3,Club Goldwand,"Kauf und Verkauf von eigenen Grundstücken, Geb...",http://www.clubgoldwand.ch/,tel:056 282 30 50,NaN,Landstrasse 6\n5415 Nussbaumen AG,https://www.google.com/maps/place/Landstrasse%...,info@clubgoldwand.ch
4,NICOLE DIEM Horgen,Detailhandel mit Brillen und anderen Sehhilfen,http://www.nicolediem.ch/,tel:044 770 10 40,NaN,Dorfplatz 3\n8810 Horgen,https://www.google.com/maps/place/Dorfplatz%20...,payments@2x.png


In [173]:
df.tail()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
996,Büro Weber AG,Detailhandel mit Möbeln,http://www.buero-weber.ch/,tel:056 622 20 50,fax:056 622 86 64,Gewerbering 23\n5610 Wohlen AG,https://www.google.com/maps/place/Gewerbering%...,NA
997,W. Kamm AG,"lnstallation von Heizungs-, Lüftungs- und Klim...",NaN,tel:071 971 10 55,NaN,Bahnhofstrasse 3\n8360 Eschlikon TG,https://www.google.com/maps/place/Bahnhofstras...,website not accessible
998,Mebrag GmbH,Herstellung von Metallkonstruktionen,http://www.mebrag.ch/,tel:032 387 25 70,NaN,Mühleweg 6\n3253 Schnottwil,https://www.google.com/maps/place/M%C3%BChlewe...,NA
999,Garage Elmiger AG,Detailhandel mit Motorenkraftstoffen (Tankstel...,NaN,NaN,NaN,Luzernerstrasse 32\n6285 Hitzkirch,https://www.google.com/maps/place/Luzernerstra...,website not accessible
1000,Sajade AG,"Sonstiger Detailhandel mit Metallwaren, Anstri...",http://www.sajade.ch/,tel:061 692 28 19,NaN,Lehenmattstr. 137\n4052 Basel,https://www.google.com/maps/place/Lehenmattstr...,NA


In [174]:
df.head() == df1.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
0,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True
3,True,True,True,True,False,True,True,True
4,True,True,True,True,False,True,True,True


In [175]:
df.tail() == df11.tail()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
996,True,True,True,True,True,True,True,True
997,True,True,False,True,False,True,True,True
998,True,True,True,True,False,True,True,True
999,True,True,False,False,False,True,True,True
1000,True,True,True,True,False,True,True,True


### End of Checkpoint-4

### xxxxxxxxxxxxxxxxxxxxxxxx Push to Github xxxxxxxxxxxxxxxxxxxxxxxxx

### Batch-12, batch_size = 1000, batch_start_index = 11001


#### !Cell is executed. Uncomment (Ctrl + A followed by Ctrl + /) and execute (Shift + Enter) again only if you want to run the batch again.

In [38]:
# # Extract info of interst
# start_time = time.time()
# batch12_initial = extract_info_of_interest(unique_company_profile_urls, batch_size = 1000, batch_start_index = 11001, sleep=5)
# end_time = time.time()
# execution_time = end_time - start_time
# print('execution time: ', execution_time)

count 1
count 2
count 3
count 4
count 5
count 6
count 7
count 8
count 9
count 10
count 11
count 12
count 13
count 14
count 15
count 16
count 17
count 18
count 19
count 20
count 21
count 22
count 23
count 24
count 25
count 26
count 27
count 28
count 29
count 30
count 31
count 32
count 33
count 34
count 35
count 36
count 37
count 38
count 39
count 40
count 41
count 42
count 43
count 44
count 45
count 46
count 47
count 48
count 49
count 50
count 51
count 52
count 53
count 54
count 55
count 56
count 57
count 58
count 59
count 60
count 61
count 62
count 63
count 64
count 65
count 66
count 67
count 68
count 69
count 70
count 71
count 72
count 73
count 74
count 75
count 76
count 77
count 78
count 79
count 80
count 81
count 82
count 83
count 84
count 85
count 86
count 87
count 88
count 89
count 90
count 91
count 92
count 93
count 94
count 95
count 96
count 97
count 98
count 99
count 100
count 101
count 102
count 103
count 104
count 105
count 106
count 107
count 108
count 109
count 110
count 11

count 832
count 833
count 834
count 835
count 836
count 837
count 838
count 839
count 840
count 841
count 842
count 843
count 844
count 845
count 846
count 847
count 848
count 849
count 850
count 851
count 852
count 853
count 854
count 855
count 856
count 857
count 858
count 859
count 860
count 861
count 862
count 863
count 864
count 865
count 866
count 867
count 868
count 869
count 870
count 871
count 872
count 873
count 874
count 875
count 876
count 877
count 878
count 879
count 880
count 881
count 882
count 883
count 884
count 885
count 886
count 887
count 888
count 889
count 890
count 891
count 892
count 893
count 894
count 895
count 896
count 897
count 898
count 899
count 900
count 901
count 902
count 903
count 904
count 905
count 906
count 907
count 908
count 909
count 910
count 911
count 912
count 913
count 914
count 915
count 916
count 917
count 918
count 919
count 920
count 921
count 922
count 923
count 924
count 925
count 926
count 927
count 928
count 929
count 930
count 931


### Checkpoint-1

In [39]:
# # Test-Block !Uncomment and execute only if running the batch again
# print(len(batch12_initial))
# print(len(batch12_initial[-1]))

1001
7


In [40]:
# # # Test-Block !Uncomment and execute only if running the batch again
# print(batch12_initial[:3])

[['Sajade AG', 'Sonstiger Detailhandel mit Metallwaren, Anstrichmitteln, Bau- und Heimwerkerbedarf', 'http://www.sajade.ch/', 'tel:061 692 28 19', 'NA', 'Lehenmattstr. 137\n4052 Basel', 'https://www.google.com/maps/place/Lehenmattstr.%20137+4052+Basel'], ['Gasthof zum Schütz', 'Bewertungen', 'NA', 'tel:031 781 01 17', 'NA', 'Bernstrasse 5\n3629 Oppligen', 'https://www.google.com/maps/place/Bernstrasse%205+3629+Oppligen'], ['HW-Handel Inh. Wirthlin', 'Sonstiger Detailhandel mit Metallwaren, Anstrichmitteln, Bau- und Heimwerkerbedarf', 'NA', 'NA', 'NA', 'Gewerbepark Bata\n4313 Möhlin', 'https://www.google.com/maps/place/Gewerbepark%20Bata+4313+M%C3%B6hlin']]


### End of Checkpoint-1

In [176]:
batch12df = pd.read_csv('batch12.csv')

In [177]:
batch12df_initial = batch12df.drop(['email'], axis = 1)

In [178]:
batch12df_initial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     1000 non-null   object
 1   industry         1001 non-null   object
 2   website          538 non-null    object
 3   telephone        864 non-null    object
 4   fax              548 non-null    object
 5   address          1000 non-null   object
 6   address_maplink  1000 non-null   object
dtypes: object(7)
memory usage: 54.9+ KB


In [179]:
batch12_initial = batch12df_initial.values.tolist()


#### Following Function Extracts emails from All Pages of a Site (Takes A Lot of Time: 7-15 sec per page)


In [180]:
# Extract emails
start_time = time.time()
pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
batch12 = extract_emails_from_allpages(batch12_initial, pattern = pattern , index=2)
end_time = time.time()
execution_time = end_time - start_time
print('execution time: ', execution_time)

C:\Users\Admin\miniconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


No of emails found:  320 ['max.muster@gmail.ch', 'info@buero-koebeli.ch', 'info@wilcowa.ch', 'info@laubi-innovent.ch', 'info@siegenthalerag.ch', 'baumann@baumann-kernbohrungen.ch', '605a7baede844d278b89dc95ae0a9123@sentry-next.wixpress.com', 'd.zimmermann@pestalozzi.ch', 'urs.wegmueller@wegmuellerag.ch', 'info@danielluderag.ch', 'info@zuegerhome.ch', 't.lieberherr@eigerdiamant.ch', 'info@halag.ch', 'dd0a55ccb8124b9c9d938e3acf41f8aa@sentry.wixpress.com', 'info@msbh.ch', 'info@bremag-platten.ch', 'info@catering-metzg.ch', 'f.odermatt@felixodermatt.ch', 'info@kontrastag.ch', 'info@afeno.ch', 'info@scheiweiler.ch', 'info@sunneberg.ch', 'info@treuhand-dubach.ch', 'info@asgal.ch', 'info@bernardfrei.ch', 'Humbel-Rheineck-1-cbcc660fdea87e5g353ee502603ef88c@2x.jpg', 'shop@regalino.ch', 'madlen.gamma@sport-auto-staub.com', 'Cornertrader-31cf1eff-ad2f306e@182w.jpg', 'shop@bluemekari.ch', 'info@goetschi-handel.ch', 'info@cassinatech.ch', 'info@weil-plattenleger.ch', 'diretta@radioticino.com', '605

### Checkpoint-2

In [181]:
print(len(batch12)) # should be same as batch_size i.e. 1000
print(len(batch12[-1])) # should be 8
print(batch12[-5:]) #should have 8 items in each list with emails or remarks added

1001
8
[['Felix Enzler Unterlagsböden AG', 'Verlegen von Fussboden', 'http://www.enzlerag.ch/', 'tel:071 433 19 55', 'fax:071 433 22 55', 'Lerchenstrasse 22a\n9304 Bernhardzell', 'https://www.google.com/maps/place/Lerchenstrasse%2022a+9304+Bernhardzell', 'website not accessible'], ['Claudio Beer', 'Schreinerarbeiten im Innenausbau', nan, 'tel:081 252 48 62', 'fax:081 253 31 49', 'Grünbergstrasse 16\n7000 Chur', 'https://www.google.com/maps/place/Gr%C3%BCnbergstrasse%2016+7000+Chur', 'website not accessible'], ['PBH IDEA AG', 'Grosshandel mit Baustoffen', 'http://www.pbh-idea.ch/', nan, nan, 'Bösch 21\n6331 Hünenberg', 'https://www.google.com/maps/place/B%C3%B6sch%2021+6331+H%C3%BCnenberg', 'NA'], ['Imwinkelried Lüftung und Klima AG', 'lnstallation von Heizungs-, Lüftungs- und Klimaanlagen', 'http://www.imwinkelried.ch/', 'tel:027 948 07 20', 'fax:027 948 07 25', 'Torweg 10\n3930 Visp', 'https://www.google.com/maps/place/Torweg%2010+3930+Visp', 'bad link'], ['Wohnbaugenoss.Reigoldswil',

### End of Checkpoint-2

In [182]:
# # Convert to Dataframe batch11
df12 = pd.DataFrame(batch12, columns=columns)


### Checkpoint-3

In [183]:
df12.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     1000 non-null   object
 1   industry         1001 non-null   object
 2   website          538 non-null    object
 3   telephone        864 non-null    object
 4   fax              548 non-null    object
 5   address          1000 non-null   object
 6   address_maplink  1000 non-null   object
 7   email_all        1001 non-null   object
dtypes: object(8)
memory usage: 62.7+ KB


In [184]:
# save as csv
df12.to_csv('batch12_all.csv', index = False)

### End of Checkpoint-3

In [185]:
#### Merge the two dataframes
df = pd.concat([df, df12])

### Checkpoint-4

In [186]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12012 entries, 0 to 1000
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     11867 non-null  object
 1   industry         11874 non-null  object
 2   website          6817 non-null   object
 3   telephone        10510 non-null  object
 4   fax              6655 non-null   object
 5   address          11867 non-null  object
 6   address_maplink  11899 non-null  object
 7   email_all        12012 non-null  object
dtypes: object(8)
memory usage: 844.6+ KB


In [187]:
df.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
0,Ringler + Strahm Storenbau AG,Sonstige Bauinstallation,http://www.ringler-strahm.ch/,tel:033 345 22 55,fax:033 345 55 19,Uetendorfstrasse 20\n3634 Thierachern,https://www.google.com/maps/place/Uetendorfstr...,info@ringler-strahm.ch
1,Grob & Partner Architektur AG,Architekturbüros,http://www.grobarchitektur.ch/,tel:081 720 02 00,fax:081 720 02 05,Bahnhofstrasse 3\n7320 Sargans,https://www.google.com/maps/place/Bahnhofstras...,605a7baede844d278b89dc95ae0a9123@sentry-next.w...
2,Lauclair AG,Schreinerarbeiten im Innenausbau,http://www.lauclair.ch/,tel:031 879 01 69,fax:031 879 20 69,Lyssstrasse 27\n3054 Schüpfen,https://www.google.com/maps/place/Lyssstrasse%...,a95fca1f1eb9fe8g2c9ead0cd9931e2a@2x.jpg
3,Club Goldwand,"Kauf und Verkauf von eigenen Grundstücken, Geb...",http://www.clubgoldwand.ch/,tel:056 282 30 50,NaN,Landstrasse 6\n5415 Nussbaumen AG,https://www.google.com/maps/place/Landstrasse%...,info@clubgoldwand.ch
4,NICOLE DIEM Horgen,Detailhandel mit Brillen und anderen Sehhilfen,http://www.nicolediem.ch/,tel:044 770 10 40,NaN,Dorfplatz 3\n8810 Horgen,https://www.google.com/maps/place/Dorfplatz%20...,payments@2x.png


In [188]:
df.tail()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
996,Felix Enzler Unterlagsböden AG,Verlegen von Fussboden,http://www.enzlerag.ch/,tel:071 433 19 55,fax:071 433 22 55,Lerchenstrasse 22a\n9304 Bernhardzell,https://www.google.com/maps/place/Lerchenstras...,website not accessible
997,Claudio Beer,Schreinerarbeiten im Innenausbau,NaN,tel:081 252 48 62,fax:081 253 31 49,Grünbergstrasse 16\n7000 Chur,https://www.google.com/maps/place/Gr%C3%BCnber...,website not accessible
998,PBH IDEA AG,Grosshandel mit Baustoffen,http://www.pbh-idea.ch/,NaN,NaN,Bösch 21\n6331 Hünenberg,https://www.google.com/maps/place/B%C3%B6sch%2...,NA
999,Imwinkelried Lüftung und Klima AG,"lnstallation von Heizungs-, Lüftungs- und Klim...",http://www.imwinkelried.ch/,tel:027 948 07 20,fax:027 948 07 25,Torweg 10\n3930 Visp,https://www.google.com/maps/place/Torweg%2010+...,bad link
1000,Wohnbaugenoss.Reigoldswil,"Kauf und Verkauf von eigenen Grundstücken, Geb...",NaN,tel:061 941 21 72,fax:061 943 10 06,Ischlagweg 19\n4418 Reigoldswil,https://www.google.com/maps/place/Ischlagweg%2...,website not accessible


In [189]:
df.head() == df1.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
0,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True
3,True,True,True,True,False,True,True,True
4,True,True,True,True,False,True,True,True


In [190]:
df.tail() == df12.tail()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
996,True,True,True,True,True,True,True,True
997,True,True,False,True,True,True,True,True
998,True,True,True,False,False,True,True,True
999,True,True,True,True,True,True,True,True
1000,True,True,False,True,True,True,True,True


### End of Checkpoint-4

### xxxxxxxxxxxxxxxxxxxxxxx Push to Github xxxxxxxxxxxxxxxxxxxxxxxx

### Batch-13, batch_size = 1000, batch_start_index = 12001


#### !Cell is executed. Uncomment (Ctrl + A followed by Ctrl + /) and execute (Shift + Enter) again only if you want to run the batch again.

In [42]:
# # Extract info of interst
# start_time = time.time()
# batch13_initial = extract_info_of_interest(unique_company_profile_urls, batch_size = 1000, batch_start_index = 12001, sleep=5)
# end_time = time.time()
# execution_time = end_time - start_time
# print('execution time: ', execution_time)

count 1
count 2
count 3
count 4
count 5
count 6
count 7
count 8
count 9
count 10
count 11
count 12
count 13
count 14
count 15
count 16
count 17
count 18
count 19
count 20
count 21
count 22
count 23
count 24
count 25
count 26
count 27
count 28
count 29
count 30
count 31
count 32
count 33
count 34
count 35
count 36
count 37
count 38
count 39
count 40
count 41
count 42
count 43
count 44
count 45
count 46
count 47
count 48
count 49
count 50
count 51
count 52
count 53
count 54
count 55
count 56
count 57
count 58
count 59
count 60
count 61
count 62
count 63
count 64
count 65
count 66
count 67
count 68
count 69
count 70
count 71
count 72
count 73
count 74
count 75
count 76
count 77
count 78
count 79
count 80
count 81
count 82
count 83
count 84
count 85
count 86
count 87
count 88
count 89
count 90
count 91
count 92
count 93
count 94
count 95
count 96
count 97
count 98
count 99
count 100
count 101
count 102
count 103
count 104
count 105
count 106
count 107
count 108
count 109
count 110
count 11

count 832
count 833
count 834
count 835
count 836
count 837
count 838
count 839
count 840
count 841
count 842
count 843
count 844
count 845
count 846
count 847
count 848
count 849
count 850
count 851
count 852
count 853
count 854
count 855
count 856
count 857
count 858
count 859
count 860
count 861
count 862
count 863
count 864
count 865
count 866
count 867
count 868
count 869
count 870
count 871
count 872
count 873
count 874
count 875
count 876
count 877
count 878
count 879
count 880
count 881
count 882
count 883
count 884
count 885
count 886
count 887
count 888
count 889
count 890
count 891
count 892
count 893
count 894
count 895
count 896
count 897
count 898
count 899
count 900
count 901
count 902
count 903
count 904
count 905
count 906
count 907
count 908
count 909
count 910
count 911
count 912
count 913
count 914
count 915
count 916
count 917
count 918
count 919
count 920
count 921
count 922
count 923
count 924
count 925
count 926
count 927
count 928
count 929
count 930
count 931


### Checkpoint-1

In [43]:
# # Test-Block !Uncomment and execute only if running the batch again
# print(len(batch13_initial))
# print(len(batch13_initial[-1]))

1001
7


In [44]:
# # # Test-Block !Uncomment and execute only if running the batch again
# print(batch13_initial[:3])

[['Wohnbaugenoss.Reigoldswil', 'Kauf und Verkauf von eigenen Grundstücken, Gebäuden und Wohnungen', 'NA', 'tel:061 941 21 72', 'fax:061 943 10 06', 'Ischlagweg 19\n4418 Reigoldswil', 'https://www.google.com/maps/place/Ischlagweg%2019+4418+Reigoldswil'], ['Silvestro Castelli GmbH', 'Werbeagenturen', 'NA', 'tel:041 203 27 70', 'NA', 'St.Karlistrasse 13b\n6004 Luzern', 'https://www.google.com/maps/place/St.Karlistrasse%2013b+6004+Luzern'], ['Erwin & Bruno Brühwiler AG', 'Schreinerarbeiten im Innenausbau', 'http://www.bruehwiler-schreinerei.ch/', 'tel:071 969 69 00', 'fax:071 966 13 05', 'Fischingerstrasse 108\n8372 Wiezikon b. Sirnach', 'https://www.google.com/maps/place/Fischingerstrasse%20108+8372+Wiezikon%20b.%20Sirnach']]


### End of Checkpoint-1

In [191]:
batch13df = pd.read_csv('batch13.csv')

In [192]:
batch13df_initial = batch13df.drop(['email'], axis = 1)

In [193]:
batch13df_initial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     980 non-null    object
 1   industry         980 non-null    object
 2   website          550 non-null    object
 3   telephone        864 non-null    object
 4   fax              554 non-null    object
 5   address          980 non-null    object
 6   address_maplink  980 non-null    object
dtypes: object(7)
memory usage: 54.9+ KB


In [194]:
batch13_initial = batch13df_initial.values.tolist()
print(batch13_initial[:3])

[['Wohnbaugenoss.Reigoldswil', 'Kauf und Verkauf von eigenen Grundstücken, Gebäuden und Wohnungen', nan, 'tel:061 941 21 72', 'fax:061 943 10 06', 'Ischlagweg 19\n4418 Reigoldswil', 'https://www.google.com/maps/place/Ischlagweg%2019+4418+Reigoldswil'], ['Silvestro Castelli GmbH', 'Werbeagenturen', nan, 'tel:041 203 27 70', nan, 'St.Karlistrasse 13b\n6004 Luzern', 'https://www.google.com/maps/place/St.Karlistrasse%2013b+6004+Luzern'], ['Erwin & Bruno Brühwiler AG', 'Schreinerarbeiten im Innenausbau', 'http://www.bruehwiler-schreinerei.ch/', 'tel:071 969 69 00', 'fax:071 966 13 05', 'Fischingerstrasse 108\n8372 Wiezikon b. Sirnach', 'https://www.google.com/maps/place/Fischingerstrasse%20108+8372+Wiezikon%20b.%20Sirnach']]


#### Following Function Extracts emails from All Pages of a Site (Takes A Lot of Time: 7-15 sec per page)


In [215]:
# Extract emails
start_time = time.time()
pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
batch13 = extract_emails_from_allpages(batch13_initial, pattern = pattern , index=2)
end_time = time.time()
execution_time = end_time - start_time
print('execution time: ', execution_time)

No of emails found:  332 ['info@bruehwiler-schreinerei.ch', 'S6P_frontal_Logo2020_frei_web@2x.png', 'info@iseleag.ch', 'info@1878.ch', 'info@garage-allemann.ch', 'info@perosa.ch', 'garage-mistail@bluewin.ch', 'hotel@kemmeriboden.ch', 'claudia@werk5.ch', 'dosahouse@kanchi.ch', 'info@garten-bieri.ch', 'st.gallen@rhomberg.com', 'info@diwefa.ch', 'info@jostservice.ch', 'info@brimato.ch', 'info@cappelletti.ch', 'service@bernard-uhren.ch', 'info@markenmentor.ch', 'info@probst-cosmetic.ch', 'info@arzhila.ch', 'info@elsener-svt.ch', 'webmaster@fclangenthal.ch', 'info@romerag.ch', '605a7baede844d278b89dc95ae0a9123@sentry-next.wixpress.com', 'info@bahnhoefli.ch', 'grogg@grogg-ag.ch', 'info@elektrofurrer.ch', 'info@jungbauspenglerei.ch', 'info@werbetafel.ch', 'info@bildhauer-meier.ch', 'p.haug@blitzgeruest.ch', 'info@raumart.ch', 'sangbarbare@hotmail.com', 'info@isofutur.ch', 'info@sammelhof.ch', 'info@autoweibel.ch', 'info@auxilium-idc.ch', 'jacques@martin.com', 'info@tschilishoestore.ch', 'info

### Checkpoint-2

In [196]:
print(len(batch13)) # should be same as batch_size i.e. 1000
print(len(batch13[-1])) # should be 8
print(batch13[-5:]) #should have 8 items in each list with emails or remarks added

1001
8
[['on3 neues wohnen AG', 'Aktivitäten der Generalunternehmen im Baugewerbe', nan, 'tel:061 511 79 00', nan, 'St. Johanns-Vorstadt 17\n4056 Basel', 'https://www.google.com/maps/place/St.%20Johanns-Vorstadt%2017+4056+Basel', 'website not accessible'], ['Due 4 You Coiffure Rüegger Judith', 'Coiffeursalons', 'http://www.4youcoiffure.ch/', 'tel:062 794 41 45', nan, 'Sennhofweg 12\n4852 Rothrist', 'https://www.google.com/maps/place/Sennhofweg%2012+4852+Rothrist', 'website not accessible'], ['Garage Auto Kunz AG', 'lnstandhaltung und Reparatur von Automobilen', 'http://www.autokunz.ch/', 'tel:056 622 13 43', 'fax:056 619 70 80', 'Bremgarterstrasse 38+75\n5610 Wohlen AG', 'https://www.google.com/maps/place/Bremgarterstrasse%2038+75+5610+Wohlen%20AG', 'alessia.calvano@autokunz.ch'], ['Duss Baumanagement AG', 'Architekturbüros', nan, 'tel:041 620 00 01', 'fax:041 620 00 02', 'Vorderlinden 2\n6374 Buochs', 'https://www.google.com/maps/place/Vorderlinden%202+6374+Buochs', 'website not acces

### End of Checkpoint-2

In [216]:
# # Convert to Dataframe batch11
df13 = pd.DataFrame(batch13, columns=columns)


ValueError: 8 columns passed, passed data had 9 columns

### Checkpoint-3

In [ ]:
df13.info()

In [ ]:
# save as csv
df13.to_csv('batch13_all.csv', index = False)

### End of Checkpoint-3

In [ ]:
#### Merge the two dataframes
df = pd.concat([df, df13])

### Checkpoint-4

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.head() == df1.head()

In [ ]:
df.tail() == df13.tail()

### End of Checkpoint-4

### xxxxxxxxxxxxxxxxxxxxxxxxx Push to Github xxxxxxxxxxxxxxxxxxxxxx

### Batch-14, batch_size = 1227, batch_start_index = 13001


#### !Cell is executed. Uncomment (Ctrl + A followed by Ctrl + /) and execute (Shift + Enter) again only if you want to run the batch again.

In [53]:
# # Extract info of interst
# start_time = time.time()
# batch14_initial = extract_info_of_interest(unique_company_profile_urls, batch_size = 1227, batch_start_index = 13001, sleep=5)
# end_time = time.time()
# execution_time = end_time - start_time
# print('execution time: ', execution_time)

count 1
count 2
count 3
count 4
count 5
count 6
count 7
count 8
count 9
count 10
count 11
count 12
count 13
count 14
count 15
count 16
count 17
count 18
count 19
count 20
count 21
count 22
count 23
count 24
count 25
count 26
count 27
count 28
count 29
count 30
count 31
count 32
count 33
count 34
count 35
count 36
count 37
count 38
count 39
count 40
count 41
count 42
count 43
count 44
count 45
count 46
count 47
count 48
count 49
count 50
count 51
count 52
count 53
count 54
count 55
count 56
count 57
count 58
count 59
count 60
count 61
count 62
count 63
count 64
count 65
count 66
count 67
count 68
count 69
count 70
count 71
count 72
count 73
count 74
count 75
count 76
count 77
count 78
count 79
count 80
count 81
count 82
count 83
count 84
count 85
count 86
count 87
count 88
count 89
count 90
count 91
count 92
count 93
count 94
count 95
count 96
count 97
count 98
count 99
count 100
count 101
count 102
count 103
count 104
count 105
count 106
count 107
count 108
count 109
count 110
count 11

count 832
count 833
count 834
count 835
count 836
count 837
count 838
count 839
count 840
count 841
count 842
count 843
count 844
count 845
count 846
count 847
count 848
count 849
count 850
count 851
count 852
count 853
count 854
count 855
count 856
count 857
count 858
count 859
count 860
count 861
count 862
count 863
count 864
count 865
count 866
count 867
count 868
count 869
count 870
count 871
count 872
count 873
count 874
count 875
count 876
count 877
count 878
count 879
count 880
count 881
count 882
count 883
count 884
count 885
count 886
count 887
count 888
count 889
count 890
count 891
count 892
count 893
count 894
count 895
count 896
count 897
count 898
count 899
count 900
count 901
count 902
count 903
count 904
count 905
count 906
count 907
count 908
count 909
count 910
count 911
count 912
count 913
count 914
count 915
count 916
count 917
count 918
count 919
count 920
count 921
count 922
count 923
count 924
count 925
count 926
count 927
count 928
count 929
count 930
count 931


### Checkpoint-1

In [62]:
# # Test_Block !Uncomment and execute only if running the batch again
# print(len(batch14_initial))
# print(len(batch14_initial[-1]))

1226
7


In [63]:
# # Test_Block !Uncomment and execute only if running the batch again
# print(batch14_initial[:3])

[['Cleanetta Trading GmbH', 'Bewertungen', 'NA', 'tel:056 633 64 22', 'fax:056 631 76 67', 'Birkenweg 5\n5615 Fahrwangen', 'https://www.google.com/maps/place/Birkenweg%205+5615+Fahrwangen'], ['Karl Barth AG', 'Herstellung von sonstigen nicht wirtschaftszweigspezifischen Maschinen a. n. g.', 'http://www.kbarth.ch/', 'tel:052 301 00 13', 'fax:052 301 00 19', 'Unterdorf 27\n8421 Dättlikon', 'https://www.google.com/maps/place/Unterdorf%2027+8421+D%C3%A4ttlikon'], ['clevergie ag', 'Elektrizitätserzeugung', 'http://www.clevergie.ch/', 'tel:062 966 00 66', 'NA', 'Möösli 307\n4954 Wyssachen', 'https://www.google.com/maps/place/M%C3%B6%C3%B6sli%20307+4954+Wyssachen']]


### End of Checkpoint-1

In [206]:
batch14df = pd.read_csv('batch14.csv')

In [207]:
batch14df_initial = batch14df.drop(['email'], axis = 1)

In [208]:
batch14df_initial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1226 entries, 0 to 1225
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     1224 non-null   object
 1   industry         1226 non-null   object
 2   website          696 non-null    object
 3   telephone        1092 non-null   object
 4   fax              672 non-null    object
 5   address          1224 non-null   object
 6   address_maplink  1224 non-null   object
dtypes: object(7)
memory usage: 67.2+ KB


In [209]:
batch14_initial = batch14df_initial.values.tolist()
print(batch14_initial[:3])

[['Cleanetta Trading GmbH', 'Bewertungen', nan, 'tel:056 633 64 22', 'fax:056 631 76 67', 'Birkenweg 5\n5615 Fahrwangen', 'https://www.google.com/maps/place/Birkenweg%205+5615+Fahrwangen'], ['Karl Barth AG', 'Herstellung von sonstigen nicht wirtschaftszweigspezifischen Maschinen a. n. g.', 'http://www.kbarth.ch/', 'tel:052 301 00 13', 'fax:052 301 00 19', 'Unterdorf 27\n8421 Dättlikon', 'https://www.google.com/maps/place/Unterdorf%2027+8421+D%C3%A4ttlikon'], ['clevergie ag', 'Elektrizitätserzeugung', 'http://www.clevergie.ch/', 'tel:062 966 00 66', nan, 'Möösli 307\n4954 Wyssachen', 'https://www.google.com/maps/place/M%C3%B6%C3%B6sli%20307+4954+Wyssachen']]


#### Following Function Extracts emails from All Pages of a Site (Takes A Lot of Time: 7-15 sec per page)


In [210]:
# Extract emails
start_time = time.time()
pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
batch14 = extract_emails_from_allpages(batch14_initial, pattern = pattern , index=2)
end_time = time.time()
execution_time = end_time - start_time
print('execution time: ', execution_time)

C:\Users\Admin\miniconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


No of emails found:  416 ['info@kbarth.ch', 'info@clevergie.ch', 'info@heeb-enzler.ch', 'info@muffuhren.ch', 'johndoe@domain.com', 'shop@bluemekari.ch', 'info@lenherr-pneuhaus.ch', 'info@hunziker-elektro.ch', 'info@s-wi.ch', 'info@malerbertogg.ch', 'igerard@bargella.ch', 'contact@sanba-events.com', 'info@perosa.ch', 'info@sg-cleaning.ch', 'info@scherzinger-databases.ch', 'info@fischer-daspasst.ch', 'info@rhykoenig.ch', 'cevian@select.com', 'sha@seminarhotelaegerisee.ch', 'info@wuethrich-schreinerei.ch', 'info@swsmedien.ch', 'info@mumenthaler-transporte.com', 'info@elke-plastic.ch', 'info@ef-carrosserie.ch', 'security@2x.svg', 'info@bobnar.ch', 'info@bmoffice.ch', 'peterka@siebdruck.org', 'info@stadtgarten-wangen.ch', 'info@allreon.ch', 'info@grgingenieure.ch', 'info@stora-gmbh.ch', 'info@dataloft.ch', '605a7baede844d278b89dc95ae0a9123@sentry-next.wixpress.com', 'info@gastroplus.ch', 'd.abatiello@gassler.ch', 'info@noser-inox.com', 'aw@kmu-nachfolgezentrum.ch', 'giardinosorengo@gmail.co

### Checkpoint-2

In [211]:
print(len(batch14)) # should be same as batch_size i.e. 1000
print(len(batch14[-1])) # should be 8
print(batch14[-5:]) #should have 8 items in each list with emails or remarks added

1226
8
[['HWB-Finger AG', 'Bauschreinerei, Fenster und Türen', 'http://www.hwb-fingerag.ch/', 'tel:076 330 44 40', nan, 'Alte Aarestrasse 3a\n3627 Heimberg', 'https://www.google.com/maps/place/Alte%20Aarestrasse%203a+3627+Heimberg', 'Info@hwb-fingerag.ch'], ['Wenger + Wirz AG', 'Elektroinstallation', nan, 'tel:052 657 41 11', 'fax:052 657 30 55', 'Schlattingerbuck\n8253 Diessenhofen', 'https://www.google.com/maps/place/Schlattingerbuck+8253+Diessenhofen', 'website not accessible'], ['AHG-Cars Biel AG', 'lnstandhaltung und Reparatur von Automobilen', 'http://www.ahg-cars.ch/', 'tel:032 328 66 03', 'fax:032 328 66 00', 'Bözingenstrasse 100\n2502 Biel/Bienne', 'https://www.google.com/maps/place/B%C3%B6zingenstrasse%20100+2502+Biel/Bienne', 'manager@agence-mint.ch'], ['Ochsner + Ochsner AG', 'Architekturbüros', 'http://www.ochsner-ochsner.ch/', 'tel:041 711 41 01', 'fax:041 711 41 03', 'Gubelstrasse 17\n6304 Zug', 'https://www.google.com/maps/place/Gubelstrasse%2017+6304+Zug', 'bad link'],

### End of Checkpoint-2

In [212]:
# # Convert to Dataframe batch11
df14 = pd.DataFrame(batch14, columns=columns)


### Checkpoint-3

In [213]:
df14.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1226 entries, 0 to 1225
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     1224 non-null   object
 1   industry         1226 non-null   object
 2   website          696 non-null    object
 3   telephone        1092 non-null   object
 4   fax              672 non-null    object
 5   address          1224 non-null   object
 6   address_maplink  1224 non-null   object
 7   email_all        1226 non-null   object
dtypes: object(8)
memory usage: 76.8+ KB


In [214]:
# save as csv
df14.to_csv('batch14_all.csv', index = False)

### End of Checkpoint-3

In [217]:
#### Merge the two dataframes
df = pd.concat([df, df14])

### Checkpoing-4

In [218]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14239 entries, 0 to 1225
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     14071 non-null  object
 1   industry         14080 non-null  object
 2   website          8063 non-null   object
 3   telephone        12466 non-null  object
 4   fax              7881 non-null   object
 5   address          14071 non-null  object
 6   address_maplink  14103 non-null  object
 7   email_all        14239 non-null  object
dtypes: object(8)
memory usage: 1001.2+ KB


In [219]:
df.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
0,Ringler + Strahm Storenbau AG,Sonstige Bauinstallation,http://www.ringler-strahm.ch/,tel:033 345 22 55,fax:033 345 55 19,Uetendorfstrasse 20\n3634 Thierachern,https://www.google.com/maps/place/Uetendorfstr...,info@ringler-strahm.ch
1,Grob & Partner Architektur AG,Architekturbüros,http://www.grobarchitektur.ch/,tel:081 720 02 00,fax:081 720 02 05,Bahnhofstrasse 3\n7320 Sargans,https://www.google.com/maps/place/Bahnhofstras...,605a7baede844d278b89dc95ae0a9123@sentry-next.w...
2,Lauclair AG,Schreinerarbeiten im Innenausbau,http://www.lauclair.ch/,tel:031 879 01 69,fax:031 879 20 69,Lyssstrasse 27\n3054 Schüpfen,https://www.google.com/maps/place/Lyssstrasse%...,a95fca1f1eb9fe8g2c9ead0cd9931e2a@2x.jpg
3,Club Goldwand,"Kauf und Verkauf von eigenen Grundstücken, Geb...",http://www.clubgoldwand.ch/,tel:056 282 30 50,NaN,Landstrasse 6\n5415 Nussbaumen AG,https://www.google.com/maps/place/Landstrasse%...,info@clubgoldwand.ch
4,NICOLE DIEM Horgen,Detailhandel mit Brillen und anderen Sehhilfen,http://www.nicolediem.ch/,tel:044 770 10 40,NaN,Dorfplatz 3\n8810 Horgen,https://www.google.com/maps/place/Dorfplatz%20...,payments@2x.png


In [220]:
df.tail()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
1221,HWB-Finger AG,"Bauschreinerei, Fenster und Türen",http://www.hwb-fingerag.ch/,tel:076 330 44 40,NaN,Alte Aarestrasse 3a\n3627 Heimberg,https://www.google.com/maps/place/Alte%20Aares...,Info@hwb-fingerag.ch
1222,Wenger + Wirz AG,Elektroinstallation,NaN,tel:052 657 41 11,fax:052 657 30 55,Schlattingerbuck\n8253 Diessenhofen,https://www.google.com/maps/place/Schlattinger...,website not accessible
1223,AHG-Cars Biel AG,lnstandhaltung und Reparatur von Automobilen,http://www.ahg-cars.ch/,tel:032 328 66 03,fax:032 328 66 00,Bözingenstrasse 100\n2502 Biel/Bienne,https://www.google.com/maps/place/B%C3%B6zinge...,manager@agence-mint.ch
1224,Ochsner + Ochsner AG,Architekturbüros,http://www.ochsner-ochsner.ch/,tel:041 711 41 01,fax:041 711 41 03,Gubelstrasse 17\n6304 Zug,https://www.google.com/maps/place/Gubelstrasse...,bad link
1225,KARL GUBLER METALLWAREN,Herstellung von Metallkonstruktionen,NaN,tel:062 723 16 72,fax:062 723 48 49,Hint.Bahnhofstrasse 3\n5034 Suhr,https://www.google.com/maps/place/Hint.Bahnhof...,website not accessible


In [221]:
df.head() == df1.head()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
0,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True
3,True,True,True,True,False,True,True,True
4,True,True,True,True,False,True,True,True


In [222]:
df.tail() == df14.tail()

,company_name,industry,website,telephone,fax,address,address_maplink,email_all
1221,True,True,True,True,False,True,True,True
1222,True,True,False,True,True,True,True,True
1223,True,True,True,True,True,True,True,True
1224,True,True,True,True,True,True,True,True
1225,True,True,False,True,True,True,True,True


In [223]:
## verify
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14239 entries, 0 to 1225
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     14071 non-null  object
 1   industry         14080 non-null  object
 2   website          8063 non-null   object
 3   telephone        12466 non-null  object
 4   fax              7881 non-null   object
 5   address          14071 non-null  object
 6   address_maplink  14103 non-null  object
 7   email_all        14239 non-null  object
dtypes: object(8)
memory usage: 1001.2+ KB


In [224]:
## Save all batches in a csv file
df.to_csv('all_batches_all.csv', index = False)

### End of Checkpoint-4

### xxxxxxxxxxxxxxxxxxxxxx Push to Github xxxxxxxxxxxxxxxxxxxxxxxxx